# XGboost testing

##Initial Setup

In [0]:
from pyspark.sql import Row,SparkSession
from pyspark.ml.linalg import Vectors, VectorUDT
from pyspark.sql.types import *
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.feature import StandardScaler
from pyspark.ml.feature import MinMaxScaler
from pyspark.mllib.evaluation import MulticlassMetrics, BinaryClassificationMetrics
from pyspark.ml.evaluation import BinaryClassificationEvaluator
import time
from pyspark.sql.functions import *
import numpy as np
import random
import datetime
import pandas as pd
import xgboost as xgb
import mlflow.xgboost
import math
import itertools

from sparkdl.xgboost import XgboostRegressor,XgboostClassifier
from sklearn.model_selection import RandomizedSearchCV

In [0]:
blob_container = "team06" # The name of your container created in https://portal.azure.com
storage_account = "apatel" # The name of your Storage account created in https://portal.azure.com
secret_scope = "team06" # The name of the scope created in your local computer using the Databricks CLI
secret_key = "team06" # The name of the secret key created in your local computer using the Databricks CLI 
blob_url = f"wasbs://{blob_container}@{storage_account}.blob.core.windows.net"
mount_path = "/mnt/mids-w261"

In [0]:
spark.catalog.clearCache()
spark.conf.set("spark.tasks.cpus","spark.executor.cores")

In [0]:
df_imported = spark.read.parquet(f'{blob_url}/full_join_mattsFeats_anandFeats_cleaned_extraFeatures_upToOD_good_v8_4-6-22')

##Set Up Feature Lists

In [0]:
# # copied from section `Flights Features to Keep`
# Features_for_flights = [ 'DISTANCE','ORI_elevation','DEST_elevation', 
#                    'prior_dep_delayed', 'previous_DEP_DELAY_NEW',
#                     'prior_arr_delayed', 'previous_ARR_DELAY_NEW', 'plane_is_here',
#                     'avg_carrier_delay_24hrs', 'flights_sch_Today_ORIGIN', 'flights_sch_Today_DEST', 'avg_ori_airport_delay_24hrs', 'daytime', 'evening',
#                     'new_england', 'mid_atlantic', 'south', 'midwest', 'southwest', 'west', 'pacific_islands', 
#                     'spring', 'winter', 'autumn',
#                     'weekend_or_holiday','prior_dep_delayed','previous_DEP_DELAY_NEW','plane_is_here','previous_ARR_DELAY_NEW','avg_carrier_delay_24hrs',
#                     'prior_arr_delayed','avg_ori_airport_delay_24hrs','depDelayPageRank','arrDelayPageRank'
#                      ]

# Features_for_weather = ['OC1_0_wind_gust_spd_rate_imp', 'AA1_1_liquid_precip', 'AA3_1_liquid_precip', 'gd1_0_sky_coverage', 'au2_4_extreme_wind_weather', 'mv1_0_sand_dust_near', 'mv1_0_thunder_rain_near', 'aw1_mw1_0_smoke_haze_dust', 'aw1_mw1_0_fog', 'aw1_mw1_0_rain_drizzle', 'aw1_mw1_0_freezing_rain_drizzle', 'aw1_mw1_0_snow', 'aw1_mw1_0_hail_or_ice', 'aw1_mw1_0_thunderstorm', 'aw1_mw1_0_tornado', 'SLP_0_avg_station_press_imp', 'WND_3_wind_speed_imp', 'CIG_0_sky_ceiling_height_imp', 'VIS_0_visibility_dist_imp', 'TMP_0_air_temperature_imp', 'DEW_0_dew_pt_temp_imp', 'MA1_0_altimeter_set_rate_imp', 'MA1_2_station_pres_rate_imp', 'GD1_3_cloud_height_imp']

# new_features_4_6 = ['avg_hourly_delay_24hr', 'avg_OD_dep_del15_24hr', 'avg_OD_delay_min_24hr', 'avg_OD_num_flights_24hr', 'depDelayPageRank_ordinal', 'arrDelayPageRank_ordinal', 'departed_for_current_aiport', 'time_inb_flight_min', 'avg_time_inb_flights_carrier_24hr', 'avg_time_inb_flights_origin_24hr', 'avg_time_inb_flights_dest_24hr', 'avg_ori_DEP_DELAY_NEW_24hr','airline_carrier_del', 'airline_carrier_del_min', 'avg_carrier_delay_over15_lastQ', 'airline_carrier_del_ordinal', 'airline_carrier_del_min_ordinal', 'origin_avg_DEP_DEL15', 'origin_avg_DEP_DELAY_NEW', 'avg_origin_delay_over15_lastQ', 'origin_del_ordinal', 'origin_del_min_ordinal', 'OD_avg_DEP_DEL15', 'OD_avg_DEP_DELAY_NEW', 'avg_OD_delay_over15_lastQ', 'OD_del_ordinal', 'OD_del_min_ordinal']

In [0]:
# # copied from section `Flights Features to Keep`
# Features_for_flights = [ 'DISTANCE','ORI_elevation','DEST_elevation', 
#                    'prior_dep_delayed', 'previous_DEP_DELAY_NEW',
#                     'prior_arr_delayed', 'previous_ARR_DELAY_NEW', 'plane_is_here',
#                     'avg_carrier_delay_24hrs', 'flights_sch_Today_ORIGIN', 'flights_sch_Today_DEST', 'avg_ori_airport_delay_24hrs', 'daytime', 'evening',
#                     'new_england', 'mid_atlantic', 'south', 'midwest', 'southwest', 'west', 'pacific_islands', 
#                     'spring', 'winter', 'autumn',
#                     'weekend_or_holiday','prior_dep_delayed','previous_DEP_DELAY_NEW','plane_is_here','previous_ARR_DELAY_NEW','avg_carrier_delay_24hrs',
#                     'prior_arr_delayed','avg_ori_airport_delay_24hrs','depDelayPageRank','arrDelayPageRank'
#                      ]

# Features_for_weather = ['OC1_0_wind_gust_spd_rate_imp', 'AA1_1_liquid_precip', 'AA3_1_liquid_precip', 'gd1_0_sky_coverage', 'au2_4_extreme_wind_weather', 'mv1_0_sand_dust_near', 'mv1_0_thunder_rain_near', 'aw1_mw1_0_smoke_haze_dust', 'aw1_mw1_0_fog', 'aw1_mw1_0_rain_drizzle', 'aw1_mw1_0_freezing_rain_drizzle', 'aw1_mw1_0_snow', 'aw1_mw1_0_hail_or_ice', 'aw1_mw1_0_thunderstorm', 'aw1_mw1_0_tornado', 'SLP_0_avg_station_press_imp', 'WND_3_wind_speed_imp', 'CIG_0_sky_ceiling_height_imp', 'VIS_0_visibility_dist_imp', 'TMP_0_air_temperature_imp', 'DEW_0_dew_pt_temp_imp', 'MA1_0_altimeter_set_rate_imp', 'MA1_2_station_pres_rate_imp', 'GD1_3_cloud_height_imp']

# new_features_4_6 = ['avg_hourly_delay_24hr', 'avg_OD_dep_del15_24hr', 'avg_OD_delay_min_24hr', 'avg_OD_num_flights_24hr', 'depDelayPageRank_ordinal', 'arrDelayPageRank_ordinal', 'departed_for_current_aiport', 'time_inb_flight_min', 'avg_time_inb_flights_carrier_24hr', 'avg_time_inb_flights_origin_24hr', 'avg_time_inb_flights_dest_24hr', 'avg_ori_DEP_DELAY_NEW_24hr','airline_carrier_del', 'airline_carrier_del_min', 'avg_carrier_delay_over15_lastQ', 'airline_carrier_del_ordinal', 'airline_carrier_del_min_ordinal', 'origin_avg_DEP_DEL15', 'origin_avg_DEP_DELAY_NEW', 'avg_origin_delay_over15_lastQ', 'origin_del_ordinal', 'origin_del_min_ordinal', 'OD_avg_DEP_DEL15', 'OD_avg_DEP_DELAY_NEW', 'avg_OD_delay_over15_lastQ', 'OD_del_ordinal', 'OD_del_min_ordinal']

In [0]:
# feature_list_full = Features_for_flights + Features_for_weather + new_features_4_6

In [0]:
# print(feature_list_full)

['DISTANCE', 'ORI_elevation', 'DEST_elevation', 'prior_dep_delayed', 'previous_DEP_DELAY_NEW', 'prior_arr_delayed', 'previous_ARR_DELAY_NEW', 'plane_is_here', 'avg_carrier_delay_24hrs', 'flights_sch_Today_ORIGIN', 'flights_sch_Today_DEST', 'avg_ori_airport_delay_24hrs', 'daytime', 'evening', 'new_england', 'mid_atlantic', 'south', 'midwest', 'southwest', 'west', 'pacific_islands', 'spring', 'winter', 'autumn', 'weekend_or_holiday', 'prior_dep_delayed', 'previous_DEP_DELAY_NEW', 'plane_is_here', 'previous_ARR_DELAY_NEW', 'avg_carrier_delay_24hrs', 'prior_arr_delayed', 'avg_ori_airport_delay_24hrs', 'depDelayPageRank', 'arrDelayPageRank', 'OC1_0_wind_gust_spd_rate_imp', 'AA1_1_liquid_precip', 'AA3_1_liquid_precip', 'gd1_0_sky_coverage', 'au2_4_extreme_wind_weather', 'mv1_0_sand_dust_near', 'mv1_0_thunder_rain_near', 'aw1_mw1_0_smoke_haze_dust', 'aw1_mw1_0_fog', 'aw1_mw1_0_rain_drizzle', 'aw1_mw1_0_freezing_rain_drizzle', 'aw1_mw1_0_snow', 'aw1_mw1_0_hail_or_ice', 'aw1_mw1_0_thunderstorm', 'aw1_mw1_0_tornado', 'SLP_0_avg_station_press_imp', 'WND_3_wind_speed_imp', 'CIG_0_sky_ceiling_height_imp', 'VIS_0_visibility_dist_imp', 'TMP_0_air_temperature_imp', 'DEW_0_dew_pt_temp_imp', 'MA1_0_altimeter_set_rate_imp', 'MA1_2_station_pres_rate_imp', 'GD1_3_cloud_height_imp', 'avg_hourly_delay_24hr', 'avg_OD_dep_del15_24hr', 'avg_OD_delay_min_24hr', 'avg_OD_num_flights_24hr', 'depDelayPageRank_ordinal', 'arrDelayPageRank_ordinal', 'departed_for_current_aiport', 'time_inb_flight_min', 'avg_time_inb_flights_carrier_24hr', 'avg_time_inb_flights_origin_24hr', 'avg_time_inb_flights_dest_24hr', 'avg_ori_DEP_DELAY_NEW_24hr', 'airline_carrier_del', 'airline_carrier_del_min', 'avg_carrier_delay_over15_lastQ', 'airline_carrier_del_ordinal', 'airline_carrier_del_min_ordinal', 'origin_avg_DEP_DEL15', 'origin_avg_DEP_DELAY_NEW', 'avg_origin_delay_over15_lastQ', 'origin_del_ordinal', 'origin_del_min_ordinal', 'OD_avg_DEP_DEL15', 'OD_avg_DEP_DELAY_NEW', 'avg_OD_delay_over15_lastQ', 'OD_del_ordinal', 'OD_del_min_ordinal']

In [0]:
# len(feature_list_full)

Out[42]: 85

## Rumi's full list

In [0]:
# copied from section `Flights Features to Keep`
features_for_flights=[ 'DISTANCE','ORI_elevation','DEST_elevation', 
                   'prior_dep_delayed', 'previous_DEP_DELAY_NEW',
                    'prior_arr_delayed', 'previous_ARR_DELAY_NEW', 'plane_is_here',
                    'avg_carrier_delay_24hrs', 'flights_sch_Today_ORIGIN', 'flights_sch_Today_DEST', 'avg_ori_airport_delay_24hrs',
                    'daytime', 'evening',
                    'mid_atlantic', 'south', 'midwest', 'southwest', 'west', 'pacific_islands', 'atlantic_islands',
                    'summer', 'winter', 'autumn',
                    'weekend_or_holiday']

# extra_features_added_april6 = ['avg_hourly_delay_24hr', 'avg_OD_dep_del15_24hr', 'avg_OD_delay_min_24hr', 'avg_OD_num_flights_24hr', 'depDelayPageRank_ordinal', 'arrDelayPageRank_ordinal', 'departed_for_current_aiport', 'time_inb_flight_min', 'avg_time_inb_flights_carrier_24hr', 'avg_time_inb_flights_origin_24hr', 'avg_time_inb_flights_dest_24hr', 'avg_ori_DEP_DELAY_NEW_24hr']

# added on april6 + april7
extra_features_added_april7 = ['avg_hourly_delay_24hr', 'avg_OD_dep_del15_24hr', 'avg_OD_delay_min_24hr', 'avg_OD_num_flights_24hr', 'depDelayPageRank_ordinal', 'arrDelayPageRank_ordinal', 'departed_for_current_aiport', 'time_inb_flight_min', 'avg_time_inb_flights_carrier_24hr', 'avg_time_inb_flights_origin_24hr', 'avg_time_inb_flights_dest_24hr', 'avg_ori_DEP_DELAY_NEW_24hr', 'airline_carrier_del', 'airline_carrier_del_min', 'avg_carrier_delay_over15_lastQ', 'airline_carrier_del_ordinal', 'airline_carrier_del_min_ordinal', 'origin_avg_DEP_DEL15', 'origin_avg_DEP_DELAY_NEW', 'avg_origin_delay_over15_lastQ', 'origin_del_ordinal', 'origin_del_min_ordinal', 'OD_avg_DEP_DEL15', 'OD_avg_DEP_DELAY_NEW', 'avg_OD_delay_over15_lastQ', 'OD_del_ordinal', 'OD_del_min_ordinal']

# one-hot encoded variables should be n-1. Eliminate one of the one-hot encoded categoricals to the all 0 case represents it.
# Dropped: 'late_night', 'new_england', 'spring'


features_for_weathers=['OC1_0_wind_gust_spd_rate_imp','AA1_1_liquid_precip','AA3_1_liquid_precip','gd1_0_sky_coverage',
                       'au2_4_extreme_wind_weather','mv1_0_sand_dust_near','mv1_0_thunder_rain_near','aw1_mw1_0_smoke_haze_dust','aw1_mw1_0_fog',
                       'aw1_mw1_0_rain_drizzle','aw1_mw1_0_freezing_rain_drizzle','aw1_mw1_0_snow','aw1_mw1_0_hail_or_ice','aw1_mw1_0_thunderstorm',
                       'aw1_mw1_0_tornado','SLP_0_avg_station_press_imp','WND_3_wind_speed_imp','CIG_0_sky_ceiling_height_imp',
                       'VIS_0_visibility_dist_imp','TMP_0_air_temperature_imp','DEW_0_dew_pt_temp_imp','MA1_0_altimeter_set_rate_imp',
                       'MA1_2_station_pres_rate_imp','GD1_3_cloud_height_imp']

features_for_pagerank = ['depDelayPageRank', 'arrDelayPageRank']

# Original total features
feature_list_full_0 = features_for_flights + features_for_weathers + features_for_pagerank


feature_list_full = ['DISTANCE', 'ORI_elevation', 'DEST_elevation', 'prior_dep_delayed', 'previous_DEP_DELAY_NEW', 'prior_arr_delayed', 'previous_ARR_DELAY_NEW', 'plane_is_here', 'avg_carrier_delay_24hrs', 'flights_sch_Today_ORIGIN', 'flights_sch_Today_DEST', 'avg_ori_airport_delay_24hrs', 'daytime', 'evening', 'mid_atlantic', 'south', 'midwest', 'southwest', 'west', 'pacific_islands', 'atlantic_islands', 'summer', 'winter', 'autumn', 'weekend_or_holiday', 'OC1_0_wind_gust_spd_rate_imp', 'AA1_1_liquid_precip', 'AA3_1_liquid_precip', 'SLP_0_avg_station_press_imp', 'WND_3_wind_speed_imp', 'CIG_0_sky_ceiling_height_imp', 'VIS_0_visibility_dist_imp', 'TMP_0_air_temperature_imp', 'DEW_0_dew_pt_temp_imp', 'MA1_0_altimeter_set_rate_imp', 'MA1_2_station_pres_rate_imp', 'GD1_3_cloud_height_imp', 'gd1_0_sky_coverage', 'au2_4_extreme_wind_weather', 'mv1_0_sand_dust_near', 'mv1_0_thunder_rain_near', 'aw1_mw1_0_smoke_haze_dust', 'aw1_mw1_0_fog', 'aw1_mw1_0_rain_drizzle', 'aw1_mw1_0_freezing_rain_drizzle', 'aw1_mw1_0_snow', 'aw1_mw1_0_hail_or_ice', 'aw1_mw1_0_thunderstorm', 'aw1_mw1_0_tornado'] + ['depDelayPageRank', 'arrDelayPageRank']

# old ten selected features(comment out)
# feature_list = ['prior_dep_delayed', 'previous_DEP_DELAY_NEW', 'plane_is_here', 'evening', 'previous_ARR_DELAY_NEW', 
#                 'avg_carrier_delay_24hrs', 'prior_arr_delayed', 'avg_ori_airport_delay_24hrs',
#                'depDelayPageRank', 'arrDelayPageRank']


# specify any outcome variables you also want to check for missing
outcomes_for_flights=['DEP_DEL15']#, 'DEP_DELAY_NEW']

# combine:
model_col_list = outcomes_for_flights + feature_list_full_0
model_col_list_april7 = outcomes_for_flights + feature_list_full_0 + extra_features_added_april7

print(model_col_list)

In [0]:
len(model_col_list)

In [0]:
list = []
list2 = []
for i in feature_list_full:
    if i in list:
        print(i)
        list2 += [i]
    if i not in list:
        list += [i]
list2

prior_dep_delayed
previous_DEP_DELAY_NEW
plane_is_here
previous_ARR_DELAY_NEW
avg_carrier_delay_24hrs
prior_arr_delayed
avg_ori_airport_delay_24hrs
Out[44]: ['prior_dep_delayed',
 'previous_DEP_DELAY_NEW',
 'plane_is_here',
 'previous_ARR_DELAY_NEW',
 'avg_carrier_delay_24hrs',
 'prior_arr_delayed',
 'avg_ori_airport_delay_24hrs']

In [0]:
for i in model_col_list:
    if i not in feature_list_full:
        print(i)

DEP_DEL15

In [0]:
len(feature_list_full)

Out[45]: 85

In [0]:
len(set(feature_list_full))

Out[46]: 78

In [0]:
def rebalanceDF(trainingDF, desired_neg_to_pos_ratio):
    original_dep_del15_count_df = trainingDF.groupby('DEP_DEL15').count()
    positiveTrainCount = original_dep_del15_count_df.filter(original_dep_del15_count_df.DEP_DEL15 ==  1).select(['count']).head()[0]
    negativeTrainCount = original_dep_del15_count_df.filter(original_dep_del15_count_df.DEP_DEL15 ==  0).select(['count']).head()[0]

    # undersampling the negative cases
    negativeDF = trainingDF.filter(trainingDF['DEP_DEL15']==0).sample(False, positiveTrainCount/negativeTrainCount*desired_neg_to_pos_ratio, seed=12345)
    positiveDF = trainingDF.filter(trainingDF['DEP_DEL15']==1)
    new_trainDF = positiveDF.union(negativeDF).cache()
    return new_trainDF

## Rumi Updated: Manual XGboost pipeline (4years)

In [0]:
# CV_name = 'train3years'
train_data = df_imported.where((col('YEAR').cast('int') < 2018))
val_data = df_imported.where((col('YEAR').cast('int') == 2018))
holdout_data = df_imported.where((col('YEAR').cast('int') == 2019))
# save_name = 'xgboost_test_preds_4_7_'+CV_name

print("Fold file will be saved as",save_name)

In [0]:
desired_neg_to_pos_ratio = 1.6

In [0]:
# prior_features = ['evening', 'prior_dep_delayed', 'previous_DEP_DELAY_NEW', 'plane_is_here', 'previous_ARR_DELAY_NEW', 'avg_carrier_delay_24hrs', 'prior_arr_delayed', 'avg_ori_airport_delay_24hrs', 'depDelayPageRank', 'arrDelayPageRank', 'daytime','aw1_mw1_0_thunderstorm','autumn','WND_3_wind_speed_imp','AA1_1_liquid_precip','GD1_3_cloud_height_imp'
# ]

# extra_features_added = ['avg_hourly_delay_24hr', 'avg_OD_dep_del15_24hr', 'avg_OD_delay_min_24hr', 'depDelayPageRank_ordinal', 'arrDelayPageRank_ordinal', 'departed_for_current_aiport', 'time_inb_flight_min', 'avg_ori_DEP_DELAY_NEW_24hr','airline_carrier_del', 'airline_carrier_del_min', 'avg_carrier_delay_over15_lastQ', 'airline_carrier_del_ordinal', 'airline_carrier_del_min_ordinal', 'origin_avg_DEP_DEL15', 'origin_avg_DEP_DELAY_NEW', 'avg_origin_delay_over15_lastQ', 'origin_del_ordinal', 'origin_del_min_ordinal', 'OD_avg_DEP_DEL15', 'OD_avg_DEP_DELAY_NEW', 'avg_OD_delay_over15_lastQ', 'OD_del_ordinal', 'OD_del_min_ordinal']
# 'daytime' is removed

# Replace with Tier1+Tier2
selected_features_tier1 = ['DEST_elevation','prior_dep_delayed','previous_DEP_DELAY_NEW','prior_arr_delayed',
                           'previous_ARR_DELAY_NEW','plane_is_here','avg_carrier_delay_24hrs',
                           'avg_ori_airport_delay_24hrs','evening','avg_hourly_delay_24hr','avg_OD_dep_del15_24hr','time_inb_flight_min',
                           'avg_ori_DEP_DELAY_NEW_24hr','OD_avg_DEP_DEL15']

selected_features_tier1_2 = ['DISTANCE','ORI_elevation','DEST_elevation','prior_dep_delayed','previous_DEP_DELAY_NEW','prior_arr_delayed',
                             'previous_ARR_DELAY_NEW','plane_is_here','avg_carrier_delay_24hrs','flights_sch_Today_ORIGIN','flights_sch_Today_DEST',
                             'avg_ori_airport_delay_24hrs','evening','pacific_islands','summer','OC1_0_wind_gust_spd_rate_imp','AA1_1_liquid_precip',
                             'aw1_mw1_0_rain_drizzle','aw1_mw1_0_snow','aw1_mw1_0_thunderstorm','WND_3_wind_speed_imp','CIG_0_sky_ceiling_height_imp',
                             'VIS_0_visibility_dist_imp','TMP_0_air_temperature_imp','DEW_0_dew_pt_temp_imp','MA1_0_altimeter_set_rate_imp','depDelayPageRank',
                             'arrDelayPageRank','avg_hourly_delay_24hr','avg_OD_dep_del15_24hr','avg_OD_delay_min_24hr','depDelayPageRank_ordinal',
                             'arrDelayPageRank_ordinal','departed_for_current_aiport','time_inb_flight_min','avg_time_inb_flights_dest_24hr',
                             'avg_ori_DEP_DELAY_NEW_24hr','airline_carrier_del','airline_carrier_del_min','airline_carrier_del_ordinal',
                             'airline_carrier_del_min_ordinal','origin_avg_DEP_DEL15','origin_avg_DEP_DELAY_NEW','OD_avg_DEP_DEL15',
                             'avg_OD_delay_over15_lastQ','OD_del_ordinal','OD_del_min_ordinal']

In [0]:
spark.catalog.clearCache()

desired_neg_to_pos_ratio = 1.6

# prior_features = ['evening', 'prior_dep_delayed', 'previous_DEP_DELAY_NEW', 'plane_is_here', 'previous_ARR_DELAY_NEW', 'avg_carrier_delay_24hrs', 'prior_arr_delayed', 'avg_ori_airport_delay_24hrs', 'depDelayPageRank', 'arrDelayPageRank', 'daytime','aw1_mw1_0_thunderstorm','autumn','WND_3_wind_speed_imp','AA1_1_liquid_precip','GD1_3_cloud_height_imp'
# ]

# extra_features_added = ['avg_hourly_delay_24hr', 'avg_OD_dep_del15_24hr', 'avg_OD_delay_min_24hr', 'depDelayPageRank_ordinal', 'arrDelayPageRank_ordinal', 'departed_for_current_aiport', 'time_inb_flight_min', 'avg_ori_DEP_DELAY_NEW_24hr','airline_carrier_del', 'airline_carrier_del_min', 'avg_carrier_delay_over15_lastQ', 'airline_carrier_del_ordinal', 'airline_carrier_del_min_ordinal', 'origin_avg_DEP_DEL15', 'origin_avg_DEP_DELAY_NEW', 'avg_origin_delay_over15_lastQ', 'origin_del_ordinal', 'origin_del_min_ordinal', 'OD_avg_DEP_DEL15', 'OD_avg_DEP_DELAY_NEW', 'avg_OD_delay_over15_lastQ', 'OD_del_ordinal', 'OD_del_min_ordinal']

feature_list = selected_features_tier1_2 
feature_list_full = feature_list

# feature_list = prior_features + extra_features_added

train = train_data
val = val_data
holdout = holdout_data

train = rebalanceDF(train, desired_neg_to_pos_ratio)

assembler = VectorAssembler().setInputCols(feature_list_full)\
                        .setOutputCol('vectorized_features')

assmb_train = assembler.transform(train)
assmb_val = assembler.transform(val)
assmb_holdout = assembler.transform(holdout)

# no normalization required

selected_train = assmb_train.select(['vectorized_features','DEP_DEL15']).withColumnRenamed('vectorized_features','features').withColumnRenamed('DEP_DEL15','label')

selected_val = assmb_val.select(['vectorized_features','DEP_DEL15']).withColumnRenamed('vectorized_features','features').withColumnRenamed('DEP_DEL15','label')  

selected_holdout = assmb_holdout.select(['vectorized_features','DEP_DEL15']).withColumnRenamed('vectorized_features','features').withColumnRenamed('DEP_DEL15','label')  

In [0]:
# from earlier testing, we determined the best params for fold 1--the rest will randomize as below.

params = {
'seed':0,
'eta':0.21,
'gamma':0.005,
'max_depth':8,
'subsample':0.565,
'lambda':1.43,
'num_workers':8,
'objective':'binary:logitraw',
'base_score':0.66,
'eval_metric':'aucpr',
'early_stopping_round':10,
'verbosity':0
}

In [0]:
# from initial testing

# spark.catalog.clearCache()
# spark.conf.set("spark.tasks.cpus","spark.executor.cores")

# import xgboost as xgb
# import mlflow.xgboost

# from sparkdl.xgboost import XgboostRegressor,XgboostClassifier
# from sklearn.model_selection import RandomizedSearchCV

# xgbr = XgboostClassifier(params=params)

In [0]:
# The test worked fine! tuning follows.

# spark.conf.set("spark.tasks.cpus","spark.executor.cores")

# model = xgbr.fit(selected_train)

In [0]:
# from initial testing

# val_predictions = model.transform(selected_val)
# holdout_predictions = model.transform(selected_holdout)

In [0]:
# from initial testing

# spark.conf.set("spark.tasks.cpus","spark.executor.cores")

# val_predictions_trim = val_predictions.select(["label","prediction"])
# metrics = MulticlassMetrics(val_predictions_trim.rdd)

# print("Validation set: Accuracy:",metrics.accuracy, "Recall:",metrics.recall(label=1),"Precision:", metrics.precision(1), "F1 Score:",metrics.fMeasure(1.0,1.0))

# holdout_predictions_trim = holdout_predictions.select(["label","prediction"])
# metrics = MulticlassMetrics(holdout_predictions_trim.rdd)

# print("Test set: Accuracy:",metrics.accuracy, "Recall:",metrics.recall(label=1),"Precision:", metrics.precision(1), "F1 Score:",metrics.fMeasure(1.0,1.0))

In [0]:
# no longer used

# param_grid = {
# 'seed':[0],
# 'eta':[0.05,0.15,0.25,0.5],
# 'gamma':[0.01],
# 'max_depth':[4,6,8],
# 'subsample':[0.5,0.8,1],
# 'lambda':[0.8,1,1.5,2],
# 'num_parallel_tree':[1,2],
# 'num_workers':[4],
# 'objective':['binary:logitraw'],
# 'base_score':[0.5],
# 'eval_metric':['aucpr'],
# 'verbosity':0
# }

In [0]:
# these are not the original search parameters, which bracketed a larger space.
# these are centered around the "consensus" values from the first set of training to eliminate any outliers,
# to eliminate cases where a model perhaps did not stumble upon the best range of values the first time around

def generate_random_configuration_honed():
    params = {'seed':0,
                'eta':random.uniform(0.15,0.3),
                'gamma':random.uniform(0.2,0.35),
                'max_depth':math.floor(random.uniform(4,6)),
                'subsample':random.uniform(0.5,0.7),
                'lambda':random.uniform(1.2,2.5),
                'num_parallel_tree':math.floor(random.uniform(4,8)),
                'num_workers':8,
                'objective':'binary:logitraw',
                'base_score':random.uniform(0.55,0.7),
                'eval_metric':'aucpr',
                'verbosity':0
              }
    return params

In [0]:
# CV_name = 'fold_1'
# train_data = df_imported.where((col('YEAR').cast('int') == 2015))
# val_data = df_imported.where((col('YEAR').cast('int') == 2016))
# holdout_data = df_imported.where((col('YEAR').cast('int') == 2019))
# save_name = 'xgboost_test_preds_4_9_'+CV_name
save_val_name = 'xgboost_val_preds_4_15_'+CV_name
save_name = 'xgboost_test_preds_4_7_'+CV_name

# train = train_data
# val = val_data
# holdout = holdout_data

# train = rebalanceDF(train, desired_neg_to_pos_ratio)

# assembler = VectorAssembler().setInputCols(feature_list_full)\
#                         .setOutputCol('vectorized_features')

# assmb_train = assembler.transform(train)
# assmb_val = assembler.transform(val)
# assmb_holdout = assembler.transform(holdout)

# selected_train = assmb_train.select(['vectorized_features','DEP_DEL15']).withColumnRenamed('vectorized_features','features').withColumnRenamed('DEP_DEL15','label')

# selected_val = assmb_val.select(['vectorized_features','DEP_DEL15']).withColumnRenamed('vectorized_features','features').withColumnRenamed('DEP_DEL15','label')  

# selected_holdout = assmb_holdout.select(['vectorized_features','DEP_DEL15']).withColumnRenamed('vectorized_features','features').withColumnRenamed('DEP_DEL15','label') 

print("Fold file will be saved as",save_name)

iterations = 1
best_score = 0
best_params = None
best_model = None


# re-generating based on best hyperparameters
params = {
'seed':0,
'random_state':42,
'eta':0.21,
'gamma':0.005,
'max_depth':8,
'subsample':0.565,
'lambda':1.43,
'num_workers':8,
'objective':'binary:logitraw',
'num_parallel_tree':4,
'base_score':0.66,
'eval_metric':'aucpr',
'early_stopping_round':10,
'verbosity':0
}

for i in range(iterations):

    # re-generating based on best hyperparameters
    # params = generate_random_configuration_honed()
    print(params)

    xgb = XgboostClassifier(**params)
    model = xgb.fit(selected_train)
    val_predictions = model.transform(selected_val)
    val_predictions.write.mode("overwrite").parquet(f"{blob_url}/{save_val_name}")
    val_predictions = val_predictions.select(["label","prediction"])
    metrics = MulticlassMetrics(val_predictions.rdd)
    f1_score = metrics.fMeasure(1.0,1.0)
    
    if f1_score > best_score:
        best_score = f1_score
        best_params = params
        best_model = model

    print("Validation set: Accuracy:",metrics.accuracy, "Recall:",metrics.recall(label=1),"Precision:", metrics.precision(1), "F1 Score:",f1_score)

print("For Fold 1:")
print("Best Results:", best_score)
print("Best params:", best_params)

best_mode1_fold_1 = best_model

predictions = best_model.transform(selected_holdout)

print("Best predictions saved as",save_name)
predictions.write.mode("overwrite").parquet(f"{blob_url}/{save_name}")

Fold file will be saved as xgboost_test_preds_4_7_train3years
{'seed': 0, 'random_state': 42, 'eta': 0.21, 'gamma': 0.005, 'max_depth': 8, 'subsample': 0.565, 'lambda': 1.43, 'num_workers': 8, 'objective': 'binary:logitraw', 'num_parallel_tree': 4, 'base_score': 0.66, 'eval_metric': 'aucpr', 'early_stopping_round': 10, 'verbosity': 0}
We recommend using 0.0 as missing value to achieve better performance, but you set missing param to be nan. In the case of missing != 0, for features sparse vector input, the inactive values will be treated as 0 instead of missing values, and the active values which are nan will be treated as missing value, and this case the input sparse vector will be densified when constructing XGBoost DMatrix, if feature sparsity is high and input dataset is large, then it may slow down performance or lead to out of memory.
The num_workers 8 set for xgboost distributed training is greater than current max number of concurrent spark task slots, you need wait until more task slots available or you need increase spark cluster workers.
Validation set: Accuracy: 0.8311178206652884 Recall: 0.5390656576370575 Precision: 0.5582669948452933 F1 Score: 0.5484983319006487
For Fold 1:
Best Results: 0.5484983319006487
Best params: {'seed': 0, 'random_state': 42, 'eta': 0.21, 'gamma': 0.005, 'max_depth': 8, 'subsample': 0.565, 'lambda': 1.43, 'num_workers': 8, 'objective': 'binary:logitraw', 'num_parallel_tree': 4, 'base_score': 0.66, 'eval_metric': 'aucpr', 'early_stopping_round': 10, 'verbosity': 0}
Best predictions saved as xgboost_test_preds_4_7_train3years

## Rumi Updated: BEST PARAMS
Best params: {'seed': 0, 'random_state': 42, 'eta': 0.21, 'gamma': 0.005, 'max_depth': 8, 'subsample': 0.565, 'lambda': 1.43, 'num_workers': 8, 'objective': 'binary:logitraw', 'num_parallel_tree': 4, 'base_score': 0.66, 'eval_metric': 'aucpr', 'early_stopping_round': 10, 'verbosity': 0}

#Thresholding

##Threshold function and determination

In [0]:
from pyspark.sql.types import DoubleType
from pyspark.sql.functions import lit, udf
import pandas as pd
import numpy as np
import os
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import ast
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta
import geopandas as gpd
import folium
import time
import hyperopt

from pyspark.sql.functions import col, max, min
from pyspark.sql.functions import col,isnan,when,count
from pyspark.sql.functions import desc
from pyspark.sql.types import DateType
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.feature import StandardScaler
from pyspark.ml.feature import MinMaxScaler
from pyspark.mllib.evaluation import MulticlassMetrics, BinaryClassificationMetrics
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.sql.functions import add_months, date_add
from pyspark.ml.classification import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV

from hyperopt import tpe, hp, fmin, STATUS_OK
from hyperopt.pyll.base import scope

from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta
from hyperopt import Trials


def ith_(v, i):
    try:
        return float(v[i])
    except ValueError:
        return None

ith = udf(ith_, DoubleType())

output = predictions.withColumn("del_prob",ith("probability", lit(1)))
display(output)

--------------------------------------------------------------------------- 
 NameError Traceback (most recent call last)
 <command-1644384869888166> in <module> 
 45 ith = udf ( ith_ , DoubleType ( ) ) 
 46 
 ---> 47 output = predictions . withColumn ( "del_prob" , ith ( "probability" , lit ( 1 ) ) ) 
 48 display ( output ) 

 NameError : name 'predictions' is not defined

In [0]:
search_center = 0.6
search_bounds = 0.2
granularity = 5
times_to_zoom = 3

for i in range(times_to_zoom):
    search_space = np.linspace(search_center - search_bounds, search_center + search_bounds, granularity)
    best_score = 0
    best_thresh = -1
    for threshold in search_space:
        test_df = output.select('label','del_prob')
        test_df = test_df.withColumn('prediction', when((col('del_prob') >= lit(threshold)), 1.0).otherwise(0.0))
        test_df = test_df.select('label','prediction')
        test_df.cache()
        test_metrics = MulticlassMetrics(test_df.rdd)
        f1_score = test_metrics.fMeasure(1.0,1.0)
        if f1_score > best_score:
            best_score = f1_score
            best_thresh = threshold
        print("threshold:",threshold,"f1 score:",f1_score)
    print("best score this level:", best_score, "at threshold", best_thresh)
    search_center = best_thresh
    search_bounds = search_bounds / 4

threshold: 0.39999999999999997 f1 score: 0.5336944133922591
threshold: 0.5 f1 score: 0.5498115951609706
threshold: 0.6 f1 score: 0.5395805581172934
threshold: 0.7 f1 score: 0.5064128532265374
threshold: 0.8 f1 score: 0.4537133140882068
best score this level: 0.5498115951609706 at threshold 0.5
threshold: 0.45 f1 score: 0.5452328070375564
threshold: 0.47500000000000003 f1 score: 0.5483104786564115
threshold: 0.5 f1 score: 0.5498115951609706
threshold: 0.525 f1 score: 0.5496681240687155
threshold: 0.55 f1 score: 0.5476851190800535
best score this level: 0.5498115951609706 at threshold 0.5
threshold: 0.4875 f1 score: 0.549254006004469
threshold: 0.49374999999999997 f1 score: 0.5495680610437389
threshold: 0.5 f1 score: 0.5498115951609706
threshold: 0.50625 f1 score: 0.5498584136014971
threshold: 0.5125 f1 score: 0.5498487273447262
best score this level: 0.5498584136014971 at threshold 0.50625

In [0]:
best_thresh

Out[86]: 0.50625

## Rumi UPDATED: BEST THRESHOLD  0.50625

In [0]:
def find_optimal_threshold(df, search_center=0.6, search_bounds=0.2, granularity=5, times_to_zoom=3):
    """Finds optimal threshold for a model based on f1 score"""
    
    def ith_(v, i):
        try:
            return float(v[i])
        except ValueError:
            return None
    
    ith = udf(ith_, DoubleType())
    output = df.withColumn("del_prob",ith("probability", lit(1)))
    
    for i in range(times_to_zoom):
        search_space = np.linspace(search_center - search_bounds, search_center + search_bounds, granularity)
        for threshold in search_space:
            best_score = 0
            best_thresh = -1
            test_df = output.select('label','del_prob')
            test_df = test_df.withColumn('prediction', when((col('del_prob') >= lit(threshold)), 1.0).otherwise(0.0))
            test_df = test_df.select('label','prediction')
            test_df.cache()
            test_metrics = MulticlassMetrics(test_df.rdd)
            f1_score = test_metrics.fMeasure(1.0,1.0)
            if f1_score > best_score:
                best_score = f1_score
                best_thresh = threshold
            print("threshold:",threshold,"f1 score:",f1_score)
        print("best score this level:", best_score, "at threshold", best_thresh)
        search_center = threshold
        search_bounds = search_bounds / 4
    
    print("overall best threshold:", best_thresh, "with f1 score", best_score)
    return best_thresh

In [0]:
CV_name = 'train3years'
save_val_name = 'xgboost_val_preds_4_15_'+CV_name
save_name = 'xgboost_test_preds_4_7_'+CV_name
best_thresh = 0.50625

preds_1 = spark.read.parquet(f'{blob_url}/{save_val_name}')
preds_1.cache()
# preds_1_thresh = find_optimal_threshold(preds_1)
preds_1_thresh = best_thresh
preds_1.unpersist()

# preds_2 = spark.read.parquet(f'{blob_url}/xgboost_val_preds_4_7_fold_2')
# preds_2.cache()
# preds_2_thresh = find_optimal_threshold(preds_2) 
# preds_2.unpersist()

# preds_3 = spark.read.parquet(f'{blob_url}/xgboost_val_preds_4_7_fold_3')
# preds_3.cache()
# preds_3_thresh = find_optimal_threshold(preds_3) 
# preds_3.unpersist()

# preds_4 = spark.read.parquet(f'{blob_url}/xgboost_val_preds_4_7_fold_4')
# preds_4.cache()
# preds_4_thresh = find_optimal_threshold(preds_4) 
# preds_4.unpersist()

# preds_5 = spark.read.parquet(f'{blob_url}/xgboost_val_preds_4_7_fold_5')
# preds_5.cache()
# preds_5_thresh = find_optimal_threshold(preds_5) 
# preds_5.unpersist()


#predictions.write.parquet(f"{blob_url}/catboost_2019_predictions_untuned_rebalanced_100")

Out[34]: DataFrame[features: vector, label: double, rawPrediction: vector, prediction: double, probability: vector]

##Re-Labeling on new thresholds

In [0]:
def infer_new_labels(df, threshold):
    """Based on input df and threshold, output df with re-inferred labels from new threshold."""
    def ith_(v, i):
        try:
            return float(v[i])
        except ValueError:
            return None
    
    ith = udf(ith_, DoubleType())
    output = df.withColumn("del_prob",ith("probability", lit(1)))
    
    metrics = MulticlassMetrics(df.rdd)
    orig_f1 = metrics.fMeasure(1.0,1.0)
    
    print("Original f1 score:",orig_f1)
    
    test_df = output.select('label','del_prob')
    test_df = test_df.withColumn('prediction', when((col('del_prob') >= lit(threshold)), 1.0).otherwise(0.0))
    test_df = test_df.select('label','prediction')
    test_df.cache()
    test_metrics = MulticlassMetrics(test_df.rdd)
    new_f1 = test_metrics.fMeasure(1.0,1.0)
    
    print("New f1 score:", new_f1)
    
    return test_df

### Apply new labels and save out files

In [0]:
CV_name = 'train3years'
save_name_reinf = 'xgboost_reinf_4_15_'+CV_name
save_name = +CV_name

'xgboost_test_preds_4_7_train3years'

## Need update
preds_1 = spark.read.parquet(f'{blob_url}/{save_name}')
display(preds_1)

features,label,rawPrediction,prediction,probability
"Map(vectorType -> sparse, length -> 47, indices -> List(0, 1, 2, 8, 9, 10, 20, 21, 22, 23, 24, 25, 26, 27, 28, 31, 32, 33, 34, 35, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46), values -> List(1430.0, 257.0, 607.0, 0.12, 12.0, 759.0, 15.0, 823.0, 16093.0, 144.0, 94.0, 10135.0, 0.0014776813333391957, 0.0014719665605529867, 0.178, 144.0, 152.0, 1.0, 50.0, 339.95, 0.17691659646166807, 14.490484031553956, 11.0, 7.0, 0.1598317560462671, 16.889589905362776, 0.14285714285714285, 1.0, 2377.0, 1.0))",0.0,"Map(vectorType -> dense, length -> 2, values -> List(1.837141752243042, -1.837141752243042))",0.0,"Map(vectorType -> dense, length -> 2, values -> List(0.8626103401184082, 0.13738968968391418))"
"Map(vectorType -> dense, length -> 47, values -> List(1430.0, 257.0, 607.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.07, 12.0, 784.0, 0.17, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 26.0, 1067.0, 16093.0, 139.0, 94.0, 10183.0, 0.0014776813333391957, 0.0014719665605529867, 0.109, 0.0, 0.0, 144.0, 152.0, 1.0, 102.0, 446.79, 6.666666666666667, 0.17691659646166807, 14.490484031553956, 11.0, 7.0, 0.1598317560462671, 16.889589905362776, 0.14285714285714285, 1.0, 2377.0, 1.0))",0.0,"Map(vectorType -> dense, length -> 2, values -> List(2.107152223587036, -2.107152223587036))",0.0,"Map(vectorType -> dense, length -> 2, values -> List(0.8915963768959045, 0.10840360075235367))"
"Map(vectorType -> dense, length -> 47, values -> List(1430.0, 257.0, 607.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.07, 12.0, 833.0, 0.08, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 21.0, 853.0, 16093.0, 139.0, 89.0, 10179.0, 0.0014776813333391957, 0.0014719665605529867, 0.131, 0.0, 0.0, 144.0, 152.0, 1.0, 55.0, 400.03, 4.666666666666667, 0.17691659646166807, 14.490484031553956, 11.0, 7.0, 0.1598317560462671, 16.889589905362776, 0.14285714285714285, 1.0, 2377.0, 1.0))",1.0,"Map(vectorType -> dense, length -> 2, values -> List(2.115831136703491, -2.115831136703491))",0.0,"Map(vectorType -> dense, length -> 2, values -> List(0.8924323916435242, 0.10756761580705643))"
"Map(vectorType -> dense, length -> 47, values -> List(1430.0, 257.0, 607.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.21, 13.0, 833.0, 0.25, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1219.0, 16093.0, 144.0, 78.0, 10166.0, 0.0014776813333391957, 0.0014719665605529867, 0.167, 1.0, 20.0, 144.0, 152.0, 1.0, 115.0, 346.15, 9.25, 0.17691659646166807, 14.490484031553956, 11.0, 7.0, 0.1598317560462671, 16.889589905362776, 0.14285714285714285, 1.0, 2377.0, 1.0))",0.0,"Map(vectorType -> dense, length -> 2, values -> List(1.3154305219650269, -1.3154305219650269))",0.0,"Map(vectorType -> dense, length -> 2, values -> List(0.7884204387664795, 0.2115795612335205))"
"Map(vectorType -> dense, length -> 47, values -> List(1430.0, 257.0, 607.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.16, 11.0, 636.0, 0.15, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 21.0, 1097.0, 16093.0, 156.0, 100.0, 10200.0, 0.0014776813333391957, 0.0014719665605529867, 0.157, 0.0, 0.0, 144.0, 152.0, 1.0, 89.0, 332.02, 29.76923076923077, 0.17691659646166807, 14.490484031553956, 11.0, 7.0, 0.1598317560462671, 16.889589905362776, 0.14285714285714285, 1.0, 2377.0, 1.0))",0.0,"Map(vectorType -> dense, length -> 2, values -> List(1.7156827449798584, -1.7156827449798584))",0.0,"Map(vectorType -> dense, length -> 2, values -> List(0.8475719094276428, 0.15242809057235718))"
"Map(vectorType -> dense, length -> 47, values -> List(1430.0, 257.0, 607.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.26, 12.0, 787.0, 0.27, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 21.0, 22000.0, 12875.0, 167.0, 111.0, 10257.0, 0.0014776813333391957, 0.0014719665605529867, 0.196, 0.0, 1.0, 144.0, 152.0, 1.0, 62.0, 429.34, 18.454545454545453, 0.17691659646166807, 14.490484031553956, 11.0, 7.0, 0.1598317560462671, 16.889589905362776, 0.14285714285714285, 1.0, 2377.0, 1.0))",0.0,"Map(vectorType -> dense, length -> 2, values -> List(1.476307988166809, -1.476307988166809))",0.0,"Map(vectorType -> dense, length -> 2, values -> List(0.81

In [0]:
CV_name = 'train3years'
save_name_reinf = 'xgboost_reinf_4_15_'+CV_name
save_name = 'xgboost_test_preds_4_7_'+CV_name


## Need update
preds_1 = spark.read.parquet(f'{blob_url}/{save_name}')
preds_1.cache()
# preds_1_reinf = infer_new_labels(df=preds_1, threshold=preds_1_thresh)
df=preds_1
threshold=preds_1_thresh


#detailed code of function "infer_new_labels". Just checking where the error ocurrs  
ith = udf(ith_, DoubleType())
output = df.withColumn("del_prob",ith("probability", lit(1)))

metrics = MulticlassMetrics(df.select('label','prediction').rdd)
orig_f1 = metrics.fMeasure(1.0,1.0)

print("Original f1 score:",orig_f1)

test_df = output.select('label','del_prob')
test_df = test_df.withColumn('prediction', when((col('del_prob') >= lit(threshold)), 1.0).otherwise(0.0))
test_df = test_df.select('label','prediction')
test_df.cache()
test_metrics = MulticlassMetrics(test_df.rdd)
new_f1 = test_metrics.fMeasure(1.0,1.0)

print("New f1 score:", new_f1)
# CV_name = 'fold_2'
# save_name = 'xgboost_reinf_4_7_'+CV_name
# preds_2 = spark.read.parquet(f'{blob_url}/xgboost_test_preds_4_7_fold_2')
# preds_2.cache()
# preds_2_reinf = infer_new_labels(df=preds_2, threshold=preds_2_thresh)
# preds_2.unpersist()
# preds_2_reinf.cache()
# preds_2_reinf.write.parquet(f"{blob_url}/{save_name}")
# preds_2_reinf.unpersist()

# CV_name = 'fold_3'
# save_name = 'xgboost_reinf_4_7_'+CV_name
# preds_3 = spark.read.parquet(f'{blob_url}/xgboost_test_preds_4_7_fold_3')
# preds_3.cache()
# preds_3_reinf = infer_new_labels(df=preds_3, threshold=preds_3_thresh)
# preds_3.unpersist()
# preds_3_reinf.cache()
# preds_3_reinf.write.parquet(f"{blob_url}/{save_name}")
# preds_3_reinf.unpersist()

# CV_name = 'fold_4'
# save_name = 'xgboost_reinf_4_7_'+CV_name
# preds_4 = spark.read.parquet(f'{blob_url}/xgboost_test_preds_4_7_fold_4')
# preds_4.cache()
# preds_4_reinf = infer_new_labels(df=preds_4, threshold=preds_4_thresh)
# preds_4.unpersist()
# preds_4_reinf.cache()
# preds_4_reinf.write.parquet(f"{blob_url}/{save_name}")
# preds_4_reinf.unpersist()

# CV_name = 'fold_5'
# save_name = 'xgboost_reinf_4_7_'+CV_name
# preds_5 = spark.read.parquet(f'{blob_url}/xgboost_test_preds_4_7_fold_5')
# preds_5.cache()
# preds_5_reinf = infer_new_labels(df=preds_5, threshold=preds_5_thresh)
# preds_5.unpersist()
# preds_5_reinf.cache()
# preds_5_reinf.write.parquet(f"{blob_url}/{save_name}")
# preds_5_reinf.unpersist()

Original f1 score: 0.549811797085343
New f1 score: 0.5498584136014971

In [0]:
print(f'3Year-Trained XGBoost model: Accuracy: {test_metrics.accuracy}, Recall: {test_metrics.recall(label=1)}, Precision: {test_metrics.precision(1)}, F1 Score: {new_f1}')

3Year-Trained XGBoost model: Accuracy: 0.832693512918883, Recall: 0.5517619652626452, Precision: 0.5479679511027256, F1 Score: 0.5498584136014971

In [0]:
df = spark.read.parquet(f'{blob_url}/{save_name}')
df.cache()
threshold=0.50625

#detailed code of function "infer_new_labels". Just checking where the error ocurrs  
ith = udf(ith_, DoubleType())
output = df.withColumn("del_prob",ith("probability", lit(1)))

metrics = MulticlassMetrics(df.select('label','prediction').rdd)
orig_f1 = metrics.fMeasure(1.0,1.0)

print("Original f1 score:",orig_f1)

test_df = output.select('label','del_prob')
test_df = test_df.withColumn('prediction', when((col('del_prob') >= lit(threshold)), 1.0).otherwise(0.0))
test_df = test_df.select('label','prediction')
test_df.cache()
test_metrics = MulticlassMetrics(test_df.rdd)
new_f1 = test_metrics.fMeasure(1.0,1.0)

print("New f1 score:", new_f1)


--------------------------------------------------------------------------- 
 NameError Traceback (most recent call last)
 <command-1644384869888177> in <module> 
 4 
 5 #detailed code of function "infer_new_labels". Just checking where the error ocurrs 
 ----> 6 ith = udf ( ith_ , DoubleType ( ) ) 
 7 output = df . withColumn ( "del_prob" , ith ( "probability" , lit ( 1 ) ) ) 
 8 

 NameError : name 'ith_' is not defined

## Rumi UPDATED: New f1 score: 0.5498584136014971

In [0]:
preds_1_reinf= test_df
preds_1.unpersist()
preds_1_reinf.cache()
preds_1_reinf.write.parquet(f"{blob_url}/{save_name_reinf}")
preds_1_reinf.unpersist()


--------------------------------------------------------------------------- 
 NameError Traceback (most recent call last)
 <command-1644384869888179> in <module> 
 ----> 1 preds_1_reinf = test_df
 2 preds_1 . unpersist ( ) 
 3 preds_1_reinf . cache ( ) 
 4 preds_1_reinf . write . parquet ( f"{blob_url}/{save_name_reinf}" ) 
 5 preds_1_reinf . unpersist ( ) 

 NameError : name 'test_df' is not defined

# Rumi UPDATED: TRAIN 4Years + TEST 1Year

In [0]:
df_imported = spark.read.parquet(f'{blob_url}/full_join_mattsFeats_anandFeats_cleaned_extraFeatures_upToOD_good_v8_4-6-22')

In [0]:
CV_name = 'train4years'
train_data = df_imported.where((col('YEAR').cast('int') <= 2018))
# val_data = df_imported.where((col('YEAR').cast('int') == 2018))

# Use as test data
holdout_data = df_imported.where((col('YEAR').cast('int') == 2019))
save_name = 'xgboost_test_preds_4_15_'+CV_name

print("File will be saved as",save_name)

File will be saved as xgboost_test_preds_4_15_train4years

## Features list
- evening is replaced with daytime 

- Feature selection
  - https://adb-731998097721284.4.azuredatabricks.net/?o=731998097721284#notebook/1731553363140738/command/1731553363156737

In [0]:
# prior_features = ['evening', 'prior_dep_delayed', 'previous_DEP_DELAY_NEW', 'plane_is_here', 'previous_ARR_DELAY_NEW', 'avg_carrier_delay_24hrs', 'prior_arr_delayed', 'avg_ori_airport_delay_24hrs', 'depDelayPageRank', 'arrDelayPageRank', 'daytime','aw1_mw1_0_thunderstorm','autumn','WND_3_wind_speed_imp','AA1_1_liquid_precip','GD1_3_cloud_height_imp'
# ]

# extra_features_added = ['avg_hourly_delay_24hr', 'avg_OD_dep_del15_24hr', 'avg_OD_delay_min_24hr', 'depDelayPageRank_ordinal', 'arrDelayPageRank_ordinal', 'departed_for_current_aiport', 'time_inb_flight_min', 'avg_ori_DEP_DELAY_NEW_24hr','airline_carrier_del', 'airline_carrier_del_min', 'avg_carrier_delay_over15_lastQ', 'airline_carrier_del_ordinal', 'airline_carrier_del_min_ordinal', 'origin_avg_DEP_DEL15', 'origin_avg_DEP_DELAY_NEW', 'avg_origin_delay_over15_lastQ', 'origin_del_ordinal', 'origin_del_min_ordinal', 'OD_avg_DEP_DEL15', 'OD_avg_DEP_DELAY_NEW', 'avg_OD_delay_over15_lastQ', 'OD_del_ordinal', 'OD_del_min_ordinal']
# 'daytime' is removed

# Replace with Tier1+Tier2
selected_features_tier1 = ['DEST_elevation','prior_dep_delayed','previous_DEP_DELAY_NEW','prior_arr_delayed',
                           'previous_ARR_DELAY_NEW','plane_is_here','avg_carrier_delay_24hrs',
                           'avg_ori_airport_delay_24hrs','evening','avg_hourly_delay_24hr','avg_OD_dep_del15_24hr','time_inb_flight_min',
                           'avg_ori_DEP_DELAY_NEW_24hr','OD_avg_DEP_DEL15']

selected_features_tier1_2_daytime = ['DISTANCE','ORI_elevation','DEST_elevation','prior_dep_delayed','previous_DEP_DELAY_NEW','prior_arr_delayed',
                             'previous_ARR_DELAY_NEW','plane_is_here','avg_carrier_delay_24hrs','flights_sch_Today_ORIGIN','flights_sch_Today_DEST',
                             'avg_ori_airport_delay_24hrs','daytime','pacific_islands','summer','OC1_0_wind_gust_spd_rate_imp','AA1_1_liquid_precip',
                             'aw1_mw1_0_rain_drizzle','aw1_mw1_0_snow','aw1_mw1_0_thunderstorm','WND_3_wind_speed_imp','CIG_0_sky_ceiling_height_imp',
                             'VIS_0_visibility_dist_imp','TMP_0_air_temperature_imp','DEW_0_dew_pt_temp_imp','MA1_0_altimeter_set_rate_imp','depDelayPageRank',
                             'arrDelayPageRank','avg_hourly_delay_24hr','avg_OD_dep_del15_24hr','avg_OD_delay_min_24hr','depDelayPageRank_ordinal',
                             'arrDelayPageRank_ordinal','departed_for_current_aiport','time_inb_flight_min','avg_time_inb_flights_dest_24hr',
                             'avg_ori_DEP_DELAY_NEW_24hr','airline_carrier_del','airline_carrier_del_min','airline_carrier_del_ordinal',
                             'airline_carrier_del_min_ordinal','origin_avg_DEP_DEL15','origin_avg_DEP_DELAY_NEW','OD_avg_DEP_DEL15',
                             'avg_OD_delay_over15_lastQ','OD_del_ordinal','OD_del_min_ordinal']

## Assemble

In [0]:
spark.catalog.clearCache()

desired_neg_to_pos_ratio = 1.6

# prior_features = ['evening', 'prior_dep_delayed', 'previous_DEP_DELAY_NEW', 'plane_is_here', 'previous_ARR_DELAY_NEW', 'avg_carrier_delay_24hrs', 'prior_arr_delayed', 'avg_ori_airport_delay_24hrs', 'depDelayPageRank', 'arrDelayPageRank', 'daytime','aw1_mw1_0_thunderstorm','autumn','WND_3_wind_speed_imp','AA1_1_liquid_precip','GD1_3_cloud_height_imp'
# ]

# extra_features_added = ['avg_hourly_delay_24hr', 'avg_OD_dep_del15_24hr', 'avg_OD_delay_min_24hr', 'depDelayPageRank_ordinal', 'arrDelayPageRank_ordinal', 'departed_for_current_aiport', 'time_inb_flight_min', 'avg_ori_DEP_DELAY_NEW_24hr','airline_carrier_del', 'airline_carrier_del_min', 'avg_carrier_delay_over15_lastQ', 'airline_carrier_del_ordinal', 'airline_carrier_del_min_ordinal', 'origin_avg_DEP_DEL15', 'origin_avg_DEP_DELAY_NEW', 'avg_origin_delay_over15_lastQ', 'origin_del_ordinal', 'origin_del_min_ordinal', 'OD_avg_DEP_DEL15', 'OD_avg_DEP_DELAY_NEW', 'avg_OD_delay_over15_lastQ', 'OD_del_ordinal', 'OD_del_min_ordinal']

feature_list = selected_features_tier1_2_daytime
feature_list_full = feature_list

# feature_list = prior_features + extra_features_added

train = train_data
# val = val_data
holdout = holdout_data

train = rebalanceDF(train, desired_neg_to_pos_ratio)

assembler = VectorAssembler().setInputCols(feature_list_full)\
                        .setOutputCol('vectorized_features')

assmb_train = assembler.transform(train)
# assmb_val = assembler.transform(val)
assmb_holdout = assembler.transform(holdout)

# no normalization required

selected_train = assmb_train.select(['vectorized_features','DEP_DEL15']).withColumnRenamed('vectorized_features','features').withColumnRenamed('DEP_DEL15','label')

# selected_val = assmb_val.select(['vectorized_features','DEP_DEL15']).withColumnRenamed('vectorized_features','features').withColumnRenamed('DEP_DEL15','label')  

selected_holdout = assmb_holdout.select(['vectorized_features','DEP_DEL15']).withColumnRenamed('vectorized_features','features').withColumnRenamed('DEP_DEL15','label')  

## Apply Best Parameters

Best params: {'seed': 0, 'random_state': 42, 'eta': 0.21, 'gamma': 0.005, 'max_depth': 8, 'subsample': 0.565, 'lambda': 1.43, 'num_workers': 8, 'objective': 'binary:logitraw', 'num_parallel_tree': 4, 'base_score': 0.66, 'eval_metric': 'aucpr', 'early_stopping_round': 10, 'verbosity': 0}

Best Threshold:

In [0]:
best_params = {'seed': 0, 'random_state': 42, 'eta': 0.21, 'gamma': 0.005, 'max_depth': 8, 'subsample': 0.565, 'lambda': 1.43, 'num_workers': 8, 'objective': 'binary:logitraw', 'num_parallel_tree': 4, 'base_score': 0.66, 'eval_metric': 'aucpr', 'early_stopping_round': 10, 'verbosity': 0}

## Train + Run Test of Single XGBoost

In [0]:
xgb = XgboostClassifier(seed=0, random_state=42, eta=0.21, gamma=0.005, max_depth=8, subsample=0.565,  reg_lambda=1.43,
                        objective='binary:logitraw', num_parallel_tree=4, base_score=0.66, eval_metric='aucpr',
                        early_stopping_round=10, verbosity=0)

model = xgb.fit(selected_train)
holdout_predictions = model.transform(selected_holdout)


We recommend using 0.0 as missing value to achieve better performance, but you set missing param to be nan. In the case of missing != 0, for features sparse vector input, the inactive values will be treated as 0 instead of missing values, and the active values which are nan will be treated as missing value, and this case the input sparse vector will be densified when constructing XGBoost DMatrix, if feature sparsity is high and input dataset is large, then it may slow down performance or lead to out of memory.

--------------------------------------------------------------------------- 
 Py4JJavaError Traceback (most recent call last)
 <command-1644384869888190> in <module> 
 3 early_stopping_round=10, verbosity=0)
 4 
 ----> 5 model = xgb . fit ( selected_train ) 
 6 holdout_predictions = model . transform ( selected_holdout ) 

 /databricks/python_shell/dbruntime/MLWorkloadsInstrumentation/_pyspark.py in patched_method (self, *args, **kwargs) 
 28 call_succeeded = False 
 29 try : 
 ---> 30 result = original_method ( self , * args , ** kwargs ) 
 31 call_succeeded = True 
 32 return result

 /databricks/spark/python/pyspark/ml/base.py in fit (self, dataset, params) 
 159 return self . copy ( params ) . _fit ( dataset ) 
 160 else : 
 --> 161 return self . _fit ( dataset ) 
 162 else : 
 163 raise ValueError("Params must be either a param map or a list/tuple of param maps, "

 /databricks/spark/python/pyspark/databricks/utils/instrumentation.py in wrapper (self, *args, **kwargs) 
 40 try : 
 41 start_time = time . time ( ) 
 ---> 42 return_val = func ( self , * args , ** kwargs ) 
 43 except Exception as e : 
 44 duration = ( time . time ( ) - start_time ) * 1000 

 /databricks/.python_edge_libs/sparkdl/xgboost/xgboost_core.py in _fit (self, dataset) 
 574 
 575 # Train on 1 remote worker, return the string of the serialized model 
 --> 576 result_ser_model_string = dataset . repartition ( 1 ) \ 
 577 . mapInPandas ( train_func , schema = 'model_string string' ) . collect ( ) [ 0 ] [ 0 ] 
 578 

 /databricks/spark/python/pyspark/sql/dataframe.py in collect (self) 
 695 # Default path used in OSS Spark / for non-DF-ACL clusters: 
 696 with SCCallSiteSync ( self . _sc ) as css : 
 --> 697 sock_info = self . _jdf . collectToPython ( ) 
 698 return list ( _load_from_socket ( sock_info , BatchedSerializer ( PickleSerializer ( ) ) ) ) 
 699 

 /databricks/spark/python/lib/py4j-0.10.9-src.zip/py4j/java_gateway.py in __call__ (self, *args) 
 1302 
 1303 answer = self . gateway_client . send_command ( command ) 
 -> 1304 return_value = get_return_value(
 1305 answer, self.gateway_client, self.target_id, self.name)
 1306 

 /databricks/spark/python/pyspark/sql/utils.py in deco (*a, **kw) 
 115 def deco ( * a , ** kw ) : 
 116 try : 
 --> 117 return f ( * a , ** kw ) 
 118 except py4j . protocol . Py4JJavaError as e : 
 119 converted = convert_exception ( e . java_exception ) 

 /databricks/spark/python/lib/py4j-0.10.9-src.zip/py4j/protocol.py in get_return_value (answer, gateway_client, target_id, name) 
 324 value = OUTPUT_CONVERTER [ type ] ( answer [ 2 : ] , gateway_client ) 
 325 if answer [ 1 ] == REFERENCE_TYPE : 
 --> 326 raise Py4JJavaError(
 327 "An error occurred while calling {0}{1}{2}.\n" . 
 328 format(target_id, ".", name), value)

 Py4JJavaError : An error occurred while calling o822.collectToPython.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 10.0 failed 4 times, most recent failure: Lost task 0.3 in stage 10.0 (TID 125) (10.139.64.9 executor 0): org.apache.spark.SparkException: Python worker exited unexpectedly (crashed)
	at org.apache.spark.api.python.BasePythonRunner$ReaderIterator$$anonfun$1.applyOrElse(PythonRunner.scala:677)
	at org.apache.spark.api.python.BasePythonRunner$ReaderIterator$$anonfun$1.applyOrElse(PythonRunner.scala:666)
	at scala.runtime.AbstractPartialFunction.apply(AbstractPartialFunction.scala:38)
	at org.apache.spark.sql.execution.python.PythonArrowOutput$$anon$1.read(PythonArrowOutput.scala:107)
	at org.apache.spark.sql.execution.python.PythonArrowOutput$$anon$1.read(PythonArrowOutput.scala:50)
	at org.apache.spark.api.python.BasePythonRunner$ReaderIterator.hasNext(PythonRunner.scala:594)
	at org.apache.spark.InterruptibleIterator.hasNext(InterruptibleIterator.scala:37)
	at scala.collection.Iterator$$anon$11.hasNext(Iterator.scala:489)
	at scala.collection.Iterator$$anon$10.hasNext(Iterator.scala:458)
	at org.apache.spark.sql.execution.collect.UnsafeRowBatchUtil

In [0]:
CV_name = 'train4years'
save_name = 'xgboost_test_preds_4_15_'+CV_name
holdout_predictions.write.mode("overwrite").parquet(f"{blob_url}/{save_name}")

--------------------------------------------------------------------------- 
 NameError Traceback (most recent call last)
 <command-1644384869888191> in <module> 
 1 CV_name = 'train4years' 
 2 save_name = 'xgboost_test_preds_4_15_' + CV_name
 ----> 3 holdout_predictions . write . mode ( "overwrite" ) . parquet ( f"{blob_url}/{save_val_name}" ) 

 NameError : name 'holdout_predictions' is not defined

In [0]:
#maxdepth=6
xgb_2 = XgboostClassifier(seed=0, random_state=42, eta=0.21, gamma=0.005, max_depth=6, subsample=0.565,  reg_lambda=1.43,
                        objective='binary:logitraw', num_parallel_tree=4, base_score=0.66, eval_metric='aucpr',
                        early_stopping_round=10, verbosity=0)

model_2 = xgb_2.fit(selected_train)
holdout_predictions_2 = model_2.transform(selected_holdout)


We recommend using 0.0 as missing value to achieve better performance, but you set missing param to be nan. In the case of missing != 0, for features sparse vector input, the inactive values will be treated as 0 instead of missing values, and the active values which are nan will be treated as missing value, and this case the input sparse vector will be densified when constructing XGBoost DMatrix, if feature sparsity is high and input dataset is large, then it may slow down performance or lead to out of memory.

--------------------------------------------------------------------------- 
 Py4JJavaError Traceback (most recent call last)
 <command-1644384869888225> in <module> 
 4 early_stopping_round=10, verbosity=0)
 5 
 ----> 6 model_2 = xgb_2 . fit ( selected_train ) 
 7 holdout_predictions_2 = model_2 . transform ( selected_holdout ) 

 /databricks/python_shell/dbruntime/MLWorkloadsInstrumentation/_pyspark.py in patched_method (self, *args, **kwargs) 
 28 call_succeeded = False 
 29 try : 
 ---> 30 result = original_method ( self , * args , ** kwargs ) 
 31 call_succeeded = True 
 32 return result

 /databricks/spark/python/pyspark/ml/base.py in fit (self, dataset, params) 
 159 return self . copy ( params ) . _fit ( dataset ) 
 160 else : 
 --> 161 return self . _fit ( dataset ) 
 162 else : 
 163 raise ValueError("Params must be either a param map or a list/tuple of param maps, "

 /databricks/spark/python/pyspark/databricks/utils/instrumentation.py in wrapper (self, *args, **kwargs) 
 40 try : 
 41 start_time = time . time ( ) 
 ---> 42 return_val = func ( self , * args , ** kwargs ) 
 43 except Exception as e : 
 44 duration = ( time . time ( ) - start_time ) * 1000 

 /databricks/.python_edge_libs/sparkdl/xgboost/xgboost_core.py in _fit (self, dataset) 
 574 
 575 # Train on 1 remote worker, return the string of the serialized model 
 --> 576 result_ser_model_string = dataset . repartition ( 1 ) \ 
 577 . mapInPandas ( train_func , schema = 'model_string string' ) . collect ( ) [ 0 ] [ 0 ] 
 578 

 /databricks/spark/python/pyspark/sql/dataframe.py in collect (self) 
 695 # Default path used in OSS Spark / for non-DF-ACL clusters: 
 696 with SCCallSiteSync ( self . _sc ) as css : 
 --> 697 sock_info = self . _jdf . collectToPython ( ) 
 698 return list ( _load_from_socket ( sock_info , BatchedSerializer ( PickleSerializer ( ) ) ) ) 
 699 

 /databricks/spark/python/lib/py4j-0.10.9-src.zip/py4j/java_gateway.py in __call__ (self, *args) 
 1302 
 1303 answer = self . gateway_client . send_command ( command ) 
 -> 1304 return_value = get_return_value(
 1305 answer, self.gateway_client, self.target_id, self.name)
 1306 

 /databricks/spark/python/pyspark/sql/utils.py in deco (*a, **kw) 
 115 def deco ( * a , ** kw ) : 
 116 try : 
 --> 117 return f ( * a , ** kw ) 
 118 except py4j . protocol . Py4JJavaError as e : 
 119 converted = convert_exception ( e . java_exception ) 

 /databricks/spark/python/lib/py4j-0.10.9-src.zip/py4j/protocol.py in get_return_value (answer, gateway_client, target_id, name) 
 324 value = OUTPUT_CONVERTER [ type ] ( answer [ 2 : ] , gateway_client ) 
 325 if answer [ 1 ] == REFERENCE_TYPE : 
 --> 326 raise Py4JJavaError(
 327 "An error occurred while calling {0}{1}{2}.\n" . 
 328 format(target_id, ".", name), value)

 Py4JJavaError : An error occurred while calling o3423.collectToPython.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 13.0 failed 4 times, most recent failure: Lost task 0.3 in stage 13.0 (TID 227) (10.139.64.10 executor 1): org.apache.spark.SparkException: Python worker exited unexpectedly (crashed)
	at org.apache.spark.api.python.BasePythonRunner$ReaderIterator$$anonfun$1.applyOrElse(PythonRunner.scala:677)
	at org.apache.spark.api.python.BasePythonRunner$ReaderIterator$$anonfun$1.applyOrElse(PythonRunner.scala:666)
	at scala.runtime.AbstractPartialFunction.apply(AbstractPartialFunction.scala:38)
	at org.apache.spark.sql.execution.python.PythonArrowOutput$$anon$1.read(PythonArrowOutput.scala:107)
	at org.apache.spark.sql.execution.python.PythonArrowOutput$$anon$1.read(PythonArrowOutput.scala:50)
	at org.apache.spark.api.python.BasePythonRunner$ReaderIterator.hasNext(PythonRunner.scala:594)
	at org.apache.spark.InterruptibleIterator.hasNext(InterruptibleIterator.scala:37)
	at scala.collection.Iterator$$anon$11.hasNext(Iterator.scala:489)
	at scala.collection.Iterator$$anon$10.hasNext(Iterator.scala:458)
	at org.apache.spark.sql.execution.collect.UnsafeRo

In [0]:
CV_name_2 = 'train4years_2'
save_name_2 = 'xgboost_test_preds_4_15_'+CV_name_2
holdout_predictions_2.write.mode("overwrite").parquet(f"{blob_url}/{save_name_2}")

--------------------------------------------------------------------------- 
 NameError Traceback (most recent call last)
 <command-1644384869888226> in <module> 
 1 CV_name_2 = 'train4years_2' 
 2 save_name_2 = 'xgboost_test_preds_4_15_' + CV_name_2
 ----> 3 holdout_predictions_2 . write . mode ( "overwrite" ) . parquet ( f"{blob_url}/{save_name_2}" ) 

 NameError : name 'holdout_predictions_2' is not defined

In [0]:
df = spark.read.parquet(f'{blob_url}/{save_name_2}')
df.cache()
threshold=0.50625

#detailed code of function "infer_new_labels". Just checking where the error ocurrs  
ith = udf(ith_, DoubleType())
output = df.withColumn("del_prob",ith("probability", lit(1)))

metrics = MulticlassMetrics(df.select('label','prediction').rdd)
orig_f1 = metrics.fMeasure(1.0,1.0)

print("Original f1 score:",orig_f1)

test_df = output.select('label','del_prob')
test_df = test_df.withColumn('prediction', when((col('del_prob') >= lit(threshold)), 1.0).otherwise(0.0))
test_df = test_df.select('label','prediction')
test_df.cache()
test_metrics = MulticlassMetrics(test_df.rdd)
new_f1 = test_metrics.fMeasure(1.0,1.0)

print("New f1 score:", new_f1)


--------------------------------------------------------------------------- 
 AnalysisException Traceback (most recent call last)
 <command-1644384869888192> in <module> 
 ----> 1 df = spark . read . parquet ( f'{blob_url}/{save_name_2}' ) 
 2 df . cache ( ) 
 3 threshold = 0.50625 
 4 
 5 #detailed code of function "infer_new_labels". Just checking where the error ocurrs 

 /databricks/spark/python/pyspark/sql/readwriter.py in parquet (self, *paths, **options) 
 483 int96RebaseMode=int96RebaseMode)
 484 
 --> 485 return self . _df ( self . _jreader . parquet ( _to_seq ( self . _spark . _sc , paths ) ) ) 
 486 
 487 def text(self, paths, wholetext=False, lineSep=None, pathGlobFilter=None,

 /databricks/spark/python/lib/py4j-0.10.9-src.zip/py4j/java_gateway.py in __call__ (self, *args) 
 1302 
 1303 answer = self . gateway_client . send_command ( command ) 
 -> 1304 return_value = get_return_value(
 1305 answer, self.gateway_client, self.target_id, self.name)
 1306 

 /databricks/spark/python/pyspark/sql/utils.py in deco (*a, **kw) 
 121 # Hide where the exception came from that shows a non-Pythonic 
 122 # JVM exception message. 
 --> 123 raise converted from None 
 124 else : 
 125 raise 

 AnalysisException : Path does not exist: wasbs://team06@apatel.blob.core.windows.net/xgboost_test_preds_4_15_train4years_2

In [0]:
print(f'4Year-Trained XGBoost model: Accuracy: {test_metrics.accuracy}, Recall: {test_metrics.recall(label=1)}, Precision: {test_metrics.precision(1)}, F1 Score: {new_f1}')

--------------------------------------------------------------------------- 
 NameError Traceback (most recent call last)
 <command-1644384869888193> in <module> 
 ----> 1 print ( f'4Year-Trained XGBoost model: Accuracy: {test_metrics.accuracy}, Recall: {test_metrics.recall(label=1)}, Precision: {test_metrics.precision(1)}, F1 Score: {new_f1}' ) 

 NameError : name 'test_metrics' is not defined

In [0]:
#maxdepth=5
xgb_3 = XgboostClassifier(seed=0, random_state=42, eta=0.21, gamma=0.005, max_depth=5, subsample=0.565,  reg_lambda=1.43,
                        objective='binary:logitraw', num_parallel_tree=4, base_score=0.66, eval_metric='aucpr',
                        early_stopping_round=10, verbosity=0)

model_3 = xgb_3.fit(selected_train)
holdout_predictions_3 = model_2.transform(selected_holdout)

We recommend using 0.0 as missing value to achieve better performance, but you set missing param to be nan. In the case of missing != 0, for features sparse vector input, the inactive values will be treated as 0 instead of missing values, and the active values which are nan will be treated as missing value, and this case the input sparse vector will be densified when constructing XGBoost DMatrix, if feature sparsity is high and input dataset is large, then it may slow down performance or lead to out of memory.

--------------------------------------------------------------------------- 
 Py4JJavaError Traceback (most recent call last)
 <command-1644384869888194> in <module> 
 4 early_stopping_round=10, verbosity=0)
 5 
 ----> 6 model_3 = xgb_3 . fit ( selected_train ) 
 7 holdout_predictions_3 = model_2 . transform ( selected_holdout ) 

 /databricks/python_shell/dbruntime/MLWorkloadsInstrumentation/_pyspark.py in patched_method (self, *args, **kwargs) 
 28 call_succeeded = False 
 29 try : 
 ---> 30 result = original_method ( self , * args , ** kwargs ) 
 31 call_succeeded = True 
 32 return result

 /databricks/spark/python/pyspark/ml/base.py in fit (self, dataset, params) 
 159 return self . copy ( params ) . _fit ( dataset ) 
 160 else : 
 --> 161 return self . _fit ( dataset ) 
 162 else : 
 163 raise ValueError("Params must be either a param map or a list/tuple of param maps, "

 /databricks/spark/python/pyspark/databricks/utils/instrumentation.py in wrapper (self, *args, **kwargs) 
 40 try : 
 41 start_time = time . time ( ) 
 ---> 42 return_val = func ( self , * args , ** kwargs ) 
 43 except Exception as e : 
 44 duration = ( time . time ( ) - start_time ) * 1000 

 /databricks/.python_edge_libs/sparkdl/xgboost/xgboost_core.py in _fit (self, dataset) 
 574 
 575 # Train on 1 remote worker, return the string of the serialized model 
 --> 576 result_ser_model_string = dataset . repartition ( 1 ) \ 
 577 . mapInPandas ( train_func , schema = 'model_string string' ) . collect ( ) [ 0 ] [ 0 ] 
 578 

 /databricks/spark/python/pyspark/sql/dataframe.py in collect (self) 
 695 # Default path used in OSS Spark / for non-DF-ACL clusters: 
 696 with SCCallSiteSync ( self . _sc ) as css : 
 --> 697 sock_info = self . _jdf . collectToPython ( ) 
 698 return list ( _load_from_socket ( sock_info , BatchedSerializer ( PickleSerializer ( ) ) ) ) 
 699 

 /databricks/spark/python/lib/py4j-0.10.9-src.zip/py4j/java_gateway.py in __call__ (self, *args) 
 1302 
 1303 answer = self . gateway_client . send_command ( command ) 
 -> 1304 return_value = get_return_value(
 1305 answer, self.gateway_client, self.target_id, self.name)
 1306 

 /databricks/spark/python/pyspark/sql/utils.py in deco (*a, **kw) 
 115 def deco ( * a , ** kw ) : 
 116 try : 
 --> 117 return f ( * a , ** kw ) 
 118 except py4j . protocol . Py4JJavaError as e : 
 119 converted = convert_exception ( e . java_exception ) 

 /databricks/spark/python/lib/py4j-0.10.9-src.zip/py4j/protocol.py in get_return_value (answer, gateway_client, target_id, name) 
 324 value = OUTPUT_CONVERTER [ type ] ( answer [ 2 : ] , gateway_client ) 
 325 if answer [ 1 ] == REFERENCE_TYPE : 
 --> 326 raise Py4JJavaError(
 327 "An error occurred while calling {0}{1}{2}.\n" . 
 328 format(target_id, ".", name), value)

 Py4JJavaError : An error occurred while calling o5091.collectToPython.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 16.0 failed 4 times, most recent failure: Lost task 0.3 in stage 16.0 (TID 329) (10.139.64.10 executor 2): org.apache.spark.SparkException: Python worker exited unexpectedly (crashed)
	at org.apache.spark.api.python.BasePythonRunner$ReaderIterator$$anonfun$1.applyOrElse(PythonRunner.scala:677)
	at org.apache.spark.api.python.BasePythonRunner$ReaderIterator$$anonfun$1.applyOrElse(PythonRunner.scala:666)
	at scala.runtime.AbstractPartialFunction.apply(AbstractPartialFunction.scala:38)
	at org.apache.spark.sql.execution.python.PythonArrowOutput$$anon$1.read(PythonArrowOutput.scala:107)
	at org.apache.spark.sql.execution.python.PythonArrowOutput$$anon$1.read(PythonArrowOutput.scala:50)
	at org.apache.spark.api.python.BasePythonRunner$ReaderIterator.hasNext(PythonRunner.scala:594)
	at org.apache.spark.InterruptibleIterator.hasNext(InterruptibleIterator.scala:37)
	at scala.collection.Iterator$$anon$11.hasNext(Iterator.scala:489)
	at scala.collection.Iterator$$anon$10.hasNext(Iterator.scala:458)
	at org.apache.spark.sql.execution.collect.UnsafeRo

In [0]:
CV_name_3 = 'train4years_3'
save_name_3 = 'xgboost_test_preds_4_15_'+CV_name_3
holdout_predictions_3.write.mode("overwrite").parquet(f"{blob_url}/{save_name_3}")

--------------------------------------------------------------------------- 
 NameError Traceback (most recent call last)
 <command-1644384869888239> in <module> 
 1 CV_name_3 = 'train4years_3' 
 2 save_name_3 = 'xgboost_test_preds_4_15_' + CV_name_3
 ----> 3 holdout_predictions_3 . write . mode ( "overwrite" ) . parquet ( f"{blob_url}/{save_name_3}" ) 

 NameError : name 'holdout_predictions_3' is not defined

In [0]:
df = spark.read.parquet(f'{blob_url}/{save_name_3}')
df.cache()
threshold=0.50625

#detailed code of function "infer_new_labels". Just checking where the error ocurrs  
ith = udf(ith_, DoubleType())
output = df.withColumn("del_prob",ith("probability", lit(1)))

metrics = MulticlassMetrics(df.select('label','prediction').rdd)
orig_f1 = metrics.fMeasure(1.0,1.0)

print("Original f1 score:",orig_f1)

test_df = output.select('label','del_prob')
test_df = test_df.withColumn('prediction', when((col('del_prob') >= lit(threshold)), 1.0).otherwise(0.0))
test_df = test_df.select('label','prediction')
test_df.cache()
test_metrics = MulticlassMetrics(test_df.rdd)
new_f1 = test_metrics.fMeasure(1.0,1.0)

print("New f1 score:", new_f1)


--------------------------------------------------------------------------- 
 AnalysisException Traceback (most recent call last)
 <command-1644384869888232> in <module> 
 ----> 1 df = spark . read . parquet ( f'{blob_url}/{save_name_3}' ) 
 2 df . cache ( ) 
 3 threshold = 0.50625 
 4 
 5 #detailed code of function "infer_new_labels". Just checking where the error ocurrs 

 /databricks/spark/python/pyspark/sql/readwriter.py in parquet (self, *paths, **options) 
 483 int96RebaseMode=int96RebaseMode)
 484 
 --> 485 return self . _df ( self . _jreader . parquet ( _to_seq ( self . _spark . _sc , paths ) ) ) 
 486 
 487 def text(self, paths, wholetext=False, lineSep=None, pathGlobFilter=None,

 /databricks/spark/python/lib/py4j-0.10.9-src.zip/py4j/java_gateway.py in __call__ (self, *args) 
 1302 
 1303 answer = self . gateway_client . send_command ( command ) 
 -> 1304 return_value = get_return_value(
 1305 answer, self.gateway_client, self.target_id, self.name)
 1306 

 /databricks/spark/python/pyspark/sql/utils.py in deco (*a, **kw) 
 121 # Hide where the exception came from that shows a non-Pythonic 
 122 # JVM exception message. 
 --> 123 raise converted from None 
 124 else : 
 125 raise 

 AnalysisException : Path does not exist: wasbs://team06@apatel.blob.core.windows.net/xgboost_test_preds_4_15_train4years_3

In [0]:
print(f'4Year-Trained XGBoost model: Accuracy: {test_metrics.accuracy}, Recall: {test_metrics.recall(label=1)}, Precision: {test_metrics.precision(1)}, F1 Score: {new_f1}')

--------------------------------------------------------------------------- 
 NameError Traceback (most recent call last)
 <command-1644384869888234> in <module> 
 ----> 1 print ( f'4Year-Trained XGBoost model: Accuracy: {test_metrics.accuracy}, Recall: {test_metrics.recall(label=1)}, Precision: {test_metrics.precision(1)}, F1 Score: {new_f1}' ) 

 NameError : name 'test_metrics' is not defined

In [0]:
#maxdepth=4
xgb_4 = XgboostClassifier(seed=0, random_state=42, eta=0.21, gamma=0.005, max_depth=4, subsample=0.565,  reg_lambda=1.43,
                        objective='binary:logitraw', num_parallel_tree=4, base_score=0.66, eval_metric='aucpr',
                        early_stopping_round=10, verbosity=0)

model_4 = xgb_4.fit(selected_train)
holdout_predictions_4 = model_4.transform(selected_holdout)

We recommend using 0.0 as missing value to achieve better performance, but you set missing param to be nan. In the case of missing != 0, for features sparse vector input, the inactive values will be treated as 0 instead of missing values, and the active values which are nan will be treated as missing value, and this case the input sparse vector will be densified when constructing XGBoost DMatrix, if feature sparsity is high and input dataset is large, then it may slow down performance or lead to out of memory.

--------------------------------------------------------------------------- 
 Py4JJavaError Traceback (most recent call last)
 <command-1644384869888229> in <module> 
 4 early_stopping_round=10, verbosity=0)
 5 
 ----> 6 model_4 = xgb_4 . fit ( selected_train ) 
 7 holdout_predictions_4 = model_4 . transform ( selected_holdout ) 

 /databricks/python_shell/dbruntime/MLWorkloadsInstrumentation/_pyspark.py in patched_method (self, *args, **kwargs) 
 28 call_succeeded = False 
 29 try : 
 ---> 30 result = original_method ( self , * args , ** kwargs ) 
 31 call_succeeded = True 
 32 return result

 /databricks/spark/python/pyspark/ml/base.py in fit (self, dataset, params) 
 159 return self . copy ( params ) . _fit ( dataset ) 
 160 else : 
 --> 161 return self . _fit ( dataset ) 
 162 else : 
 163 raise ValueError("Params must be either a param map or a list/tuple of param maps, "

 /databricks/spark/python/pyspark/databricks/utils/instrumentation.py in wrapper (self, *args, **kwargs) 
 40 try : 
 41 start_time = time . time ( ) 
 ---> 42 return_val = func ( self , * args , ** kwargs ) 
 43 except Exception as e : 
 44 duration = ( time . time ( ) - start_time ) * 1000 

 /databricks/.python_edge_libs/sparkdl/xgboost/xgboost_core.py in _fit (self, dataset) 
 574 
 575 # Train on 1 remote worker, return the string of the serialized model 
 --> 576 result_ser_model_string = dataset . repartition ( 1 ) \ 
 577 . mapInPandas ( train_func , schema = 'model_string string' ) . collect ( ) [ 0 ] [ 0 ] 
 578 

 /databricks/spark/python/pyspark/sql/dataframe.py in collect (self) 
 695 # Default path used in OSS Spark / for non-DF-ACL clusters: 
 696 with SCCallSiteSync ( self . _sc ) as css : 
 --> 697 sock_info = self . _jdf . collectToPython ( ) 
 698 return list ( _load_from_socket ( sock_info , BatchedSerializer ( PickleSerializer ( ) ) ) ) 
 699 

 /databricks/spark/python/lib/py4j-0.10.9-src.zip/py4j/java_gateway.py in __call__ (self, *args) 
 1302 
 1303 answer = self . gateway_client . send_command ( command ) 
 -> 1304 return_value = get_return_value(
 1305 answer, self.gateway_client, self.target_id, self.name)
 1306 

 /databricks/spark/python/pyspark/sql/utils.py in deco (*a, **kw) 
 115 def deco ( * a , ** kw ) : 
 116 try : 
 --> 117 return f ( * a , ** kw ) 
 118 except py4j . protocol . Py4JJavaError as e : 
 119 converted = convert_exception ( e . java_exception ) 

 /databricks/spark/python/lib/py4j-0.10.9-src.zip/py4j/protocol.py in get_return_value (answer, gateway_client, target_id, name) 
 324 value = OUTPUT_CONVERTER [ type ] ( answer [ 2 : ] , gateway_client ) 
 325 if answer [ 1 ] == REFERENCE_TYPE : 
 --> 326 raise Py4JJavaError(
 327 "An error occurred while calling {0}{1}{2}.\n" . 
 328 format(target_id, ".", name), value)

 Py4JJavaError : An error occurred while calling o7643.collectToPython.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 19.0 failed 4 times, most recent failure: Lost task 0.3 in stage 19.0 (TID 431) (10.139.64.9 executor 0): org.apache.spark.SparkException: Python worker exited unexpectedly (crashed)
	at org.apache.spark.api.python.BasePythonRunner$ReaderIterator$$anonfun$1.applyOrElse(PythonRunner.scala:677)
	at org.apache.spark.api.python.BasePythonRunner$ReaderIterator$$anonfun$1.applyOrElse(PythonRunner.scala:666)
	at scala.runtime.AbstractPartialFunction.apply(AbstractPartialFunction.scala:38)
	at org.apache.spark.sql.execution.python.PythonArrowOutput$$anon$1.read(PythonArrowOutput.scala:107)
	at org.apache.spark.sql.execution.python.PythonArrowOutput$$anon$1.read(PythonArrowOutput.scala:50)
	at org.apache.spark.api.python.BasePythonRunner$ReaderIterator.hasNext(PythonRunner.scala:594)
	at org.apache.spark.InterruptibleIterator.hasNext(InterruptibleIterator.scala:37)
	at scala.collection.Iterator$$anon$11.hasNext(Iterator.scala:489)
	at scala.collection.Iterator$$anon$10.hasNext(Iterator.scala:458)
	at org.apache.spark.sql.execution.collect.UnsafeRow

In [0]:
CV_name_4 = 'train4years_4'
save_name_4 = 'xgboost_test_preds_4_15_'+CV_name_4
holdout_predictions_4.write.mode("overwrite").parquet(f"{blob_url}/{save_name_4}")

--------------------------------------------------------------------------- 
 NameError Traceback (most recent call last)
 <command-1644384869888231> in <module> 
 1 CV_name_4 = 'train4years_4' 
 2 save_name_4 = 'xgboost_test_preds_4_15_' + CV_name_4
 ----> 3 holdout_predictions_4 . write . mode ( "overwrite" ) . parquet ( f"{blob_url}/{save_name_4}" ) 

 NameError : name 'holdout_predictions_4' is not defined

In [0]:
df = spark.read.parquet(f'{blob_url}/{save_name_4}')
df.cache()
threshold=0.50625

#detailed code of function "infer_new_labels". Just checking where the error ocurrs  
ith = udf(ith_, DoubleType())
output = df.withColumn("del_prob",ith("probability", lit(1)))

metrics = MulticlassMetrics(df.select('label','prediction').rdd)
orig_f1 = metrics.fMeasure(1.0,1.0)

print("Original f1 score:",orig_f1)

test_df = output.select('label','del_prob')
test_df = test_df.withColumn('prediction', when((col('del_prob') >= lit(threshold)), 1.0).otherwise(0.0))
test_df = test_df.select('label','prediction')
test_df.cache()
test_metrics = MulticlassMetrics(test_df.rdd)
new_f1 = test_metrics.fMeasure(1.0,1.0)

print("New f1 score:", new_f1)


--------------------------------------------------------------------------- 
 AnalysisException Traceback (most recent call last)
 <command-1644384869888240> in <module> 
 ----> 1 df = spark . read . parquet ( f'{blob_url}/{save_name_4}' ) 
 2 df . cache ( ) 
 3 threshold = 0.50625 
 4 
 5 #detailed code of function "infer_new_labels". Just checking where the error ocurrs 

 /databricks/spark/python/pyspark/sql/readwriter.py in parquet (self, *paths, **options) 
 483 int96RebaseMode=int96RebaseMode)
 484 
 --> 485 return self . _df ( self . _jreader . parquet ( _to_seq ( self . _spark . _sc , paths ) ) ) 
 486 
 487 def text(self, paths, wholetext=False, lineSep=None, pathGlobFilter=None,

 /databricks/spark/python/lib/py4j-0.10.9-src.zip/py4j/java_gateway.py in __call__ (self, *args) 
 1302 
 1303 answer = self . gateway_client . send_command ( command ) 
 -> 1304 return_value = get_return_value(
 1305 answer, self.gateway_client, self.target_id, self.name)
 1306 

 /databricks/spark/python/pyspark/sql/utils.py in deco (*a, **kw) 
 121 # Hide where the exception came from that shows a non-Pythonic 
 122 # JVM exception message. 
 --> 123 raise converted from None 
 124 else : 
 125 raise 

 AnalysisException : Path does not exist: wasbs://team06@apatel.blob.core.windows.net/xgboost_test_preds_4_15_train4years_4

In [0]:
print(f'4Year-Trained XGBoost model: Accuracy: {test_metrics.accuracy}, Recall: {test_metrics.recall(label=1)}, Precision: {test_metrics.precision(1)}, F1 Score: {new_f1}')

--------------------------------------------------------------------------- 
 NameError Traceback (most recent call last)
 <command-1644384869888241> in <module> 
 ----> 1 print ( f'4Year-Trained XGBoost model: Accuracy: {test_metrics.accuracy}, Recall: {test_metrics.recall(label=1)}, Precision: {test_metrics.precision(1)}, F1 Score: {new_f1}' ) 

 NameError : name 'test_metrics' is not defined

## Random Forest

- Best params: maxDepth=8, maxBins=64, minInstancesPerNode=10, numTrees=100

In [0]:
import pandas as pd
import numpy as np
import os
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import ast
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta
import geopandas as gpd
import folium
import time
import hyperopt

from pyspark.sql.functions import col, max, min
from pyspark.sql.functions import col,isnan,when,count
from pyspark.sql.functions import desc
from pyspark.sql.types import DateType
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.feature import StandardScaler
from pyspark.ml.feature import MinMaxScaler
from pyspark.mllib.evaluation import MulticlassMetrics, BinaryClassificationMetrics
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.sql.functions import add_months, date_add
from pyspark.ml.classification import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV

from hyperopt import tpe, hp, fmin, STATUS_OK
from hyperopt.pyll.base import scope

from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta
from hyperopt import Trials



In [0]:
rf = RandomForestClassifier(featuresCol = 'features', labelCol = 'label', maxDepth=8, maxBins=64, minInstancesPerNode=10, numTrees=100, seed=0)

model_rf = rf.fit(selected_train)
holdout_predictions_rf = model_rf.transform(selected_holdout)

In [0]:
save_name_rf = 'randomforest_test_preds_4_15_'+CV_name
holdout_predictions_rf.write.mode("overwrite").parquet(f"{blob_url}/{save_name_rf}")

## Voting and Test Set Performance

#Voting and Test Set Performance

In [0]:
# add index to each results column for joining

from pyspark.sql.functions import desc, row_number, monotonically_increasing_id
from pyspark.sql.window import Window

# read in files

preds_1_reinf = spark.read.parquet(f'{blob_url}/{save_name_reinf}')
# preds_2_reinf = spark.read.parquet(f'{blob_url}/xgboost_reinf_4_7_fold_2')
# preds_3_reinf = spark.read.parquet(f'{blob_url}/xgboost_reinf_4_7_fold_3')
# preds_4_reinf = spark.read.parquet(f'{blob_url}/xgboost_reinf_4_7_fold_4')
# preds_5_reinf = spark.read.parquet(f'{blob_url}/xgboost_reinf_4_7_fold_5')

# verify lengths

preds_1_reinf.count()
# preds_2_reinf.count()
# preds_3_reinf.count()
# preds_4_reinf.count()
# preds_5_reinf.count()

df_with_seq_id = df.withColumn

preds_1_reinf = preds_1_reinf.withColumn('index', row_number().over(Window.orderBy(monotonically_increasing_id())) - 1)
# preds_2_reinf = preds_2_reinf.withColumn('index', row_number().over(Window.orderBy(monotonically_increasing_id())) - 1)
# preds_3_reinf = preds_3_reinf.withColumn('index', row_number().over(Window.orderBy(monotonically_increasing_id())) - 1)
# preds_4_reinf = preds_4_reinf.withColumn('index', row_number().over(Window.orderBy(monotonically_increasing_id())) - 1)
# preds_5_reinf = preds_5_reinf.withColumn('index', row_number().over(Window.orderBy(monotonically_increasing_id())) - 1)

--------------------------------------------------------------------------- 
 AnalysisException Traceback (most recent call last)
 <command-1644384869885959> in <module> 
 6 # read in files 
 7 
 ----> 8 preds_1_reinf = spark . read . parquet ( f'{blob_url}/{save_name_reinf}' ) 
 9 # preds_2_reinf = spark.read.parquet(f'{blob_url}/xgboost_reinf_4_7_fold_2') 
 10 # preds_3_reinf = spark.read.parquet(f'{blob_url}/xgboost_reinf_4_7_fold_3') 

 /databricks/spark/python/pyspark/sql/readwriter.py in parquet (self, *paths, **options) 
 483 int96RebaseMode=int96RebaseMode)
 484 
 --> 485 return self . _df ( self . _jreader . parquet ( _to_seq ( self . _spark . _sc , paths ) ) ) 
 486 
 487 def text(self, paths, wholetext=False, lineSep=None, pathGlobFilter=None,

 /databricks/spark/python/lib/py4j-0.10.9-src.zip/py4j/java_gateway.py in __call__ (self, *args) 
 1302 
 1303 answer = self . gateway_client . send_command ( command ) 
 -> 1304 return_value = get_return_value(
 1305 answer, self.gateway_client, self.target_id, self.name)
 1306 

 /databricks/spark/python/pyspark/sql/utils.py in deco (*a, **kw) 
 121 # Hide where the exception came from that shows a non-Pythonic 
 122 # JVM exception message. 
 --> 123 raise converted from None 
 124 else : 
 125 raise 

 AnalysisException : Path does not exist: wasbs://team06@apatel.blob.core.windows.net/xgboost_reinf_4_15_train3years

In [0]:
all_preds = preds_1_reinf.withColumnRenamed("prediction","pred_1")
# all_preds = all_preds.join(preds_2_reinf.select("index","prediction"), ['index']).withColumnRenamed("prediction","pred_2")
# all_preds = all_preds.join(preds_3_reinf.select("index","prediction"), ['index']).withColumnRenamed("prediction","pred_3")
# all_preds = all_preds.join(preds_4_reinf.select("index","prediction"), ['index']).withColumnRenamed("prediction","pred_4")
# all_preds = all_preds.join(preds_5_reinf.select("index","prediction"), ['index']).withColumnRenamed("prediction","pred_5")

all_preds.cache()

display(all_preds)

--------------------------------------------------------------------------- 
 NameError Traceback (most recent call last)
 <command-1644384869885960> in <module> 
 ----> 1 all_preds = preds_1_reinf . withColumnRenamed ( "prediction" , "pred_1" ) 
 2 # all_preds = all_preds.join(preds_2_reinf.select("index","prediction"), ['index']).withColumnRenamed("prediction","pred_2") 
 3 # all_preds = all_preds.join(preds_3_reinf.select("index","prediction"), ['index']).withColumnRenamed("prediction","pred_3") 
 4 # all_preds = all_preds.join(preds_4_reinf.select("index","prediction"), ['index']).withColumnRenamed("prediction","pred_4") 
 5 # all_preds = all_preds.join(preds_5_reinf.select("index","prediction"), ['index']).withColumnRenamed("prediction","pred_5") 

 NameError : name 'preds_1_reinf' is not defined

In [0]:
# all_preds = all_preds.withColumn("weighted_pred", lit(0.10)*col('pred_1') + lit(0.15)*col('pred_2') + lit(0.2)*col('pred_3') + lit(0.35)*col('pred_4')+ lit(0.2)*col('pred_5'))

# all_preds = all_preds.withColumn("prediction", (col("weighted_pred") >= lit(0.5)).cast('int'))

# all_preds.cache()

# display(all_preds)

In [0]:
all_preds.write.parquet(f"{blob_url}/xgboost_no_folds_no_weighted_preds_0415")

--------------------------------------------------------------------------- 
 NameError Traceback (most recent call last)
 <command-1644384869885962> in <module> 
 ----> 1 all_preds . write . parquet ( f"{blob_url}/xgboost_no_folds_no_weighted_preds_0415" ) 

 NameError : name 'all_preds' is not defined

In [0]:
test_df = all_preds.select('prediction','label')

test_metrics = MulticlassMetrics(test_df.rdd)
f1_score = test_metrics.fMeasure(1.0,1.0)

print("Test set (2019) f1 score:", f1_score)
print("Hurray! (we hope)...")

--------------------------------------------------------------------------- 
 NameError Traceback (most recent call last)
 <command-1644384869885963> in <module> 
 ----> 1 test_df = all_preds . select ( 'prediction' , 'label' ) 
 2 
 3 test_metrics = MulticlassMetrics ( test_df . rdd ) 
 4 f1_score = test_metrics . fMeasure ( 1.0 , 1.0 ) 
 5 

 NameError : name 'all_preds' is not defined

### Looped inference 2019

Loop through the 2019 test data and add inference for each of the 5 CV fold models to a dataframe to ensure the order of the predictions is kept.

15 columns. Need raw probabilities for each, label for each, prediction for each. Including an ID.

Thresholding & Voting XGBoost for threshold has thresholding in it. Notebook: https://adb-731998097721284.4.azuredatabricks.net/?o=731998097721284#notebook/1858507102384751/command/1858507102384884

- Do thresholding next on it.

In [0]:
# rename the xgb models
best_model_fold_1 = best_mode1_fold_1
best_model_fold_2 = best_mode1_fold_2
best_model_fold_3 = best_mode1_fold_3
best_model_fold_4 = best_mode1_fold_4
best_model_fold_5 = best_mode1_fold_5

In [0]:
from pyspark.sql.functions import desc, row_number, monotonically_increasing_id
from pyspark.sql.window import Window

In [0]:
# prepare test data for 2019
test_data_2019 = holdout_data # See cmd 26, line 4.
# test_data_2019 = df_imported.where((col('YEAR').cast('int') == 2019))

assembler = VectorAssembler().setInputCols(feature_list_full)\
                        .setOutputCol('vectorized_features')

assmb_test2019 = assembler.transform(test_data_2019)

selected_test2019 = assmb_test2019.select(['vectorized_features','DEP_DEL15']).withColumnRenamed('vectorized_features','features').withColumnRenamed('DEP_DEL15','label')

selected_test2019_withID = selected_test2019.withColumn("_id", row_number().over(Window.orderBy(monotonically_increasing_id()))).cache()

In [0]:
num_flights_test2019 = selected_test2019_withID.count()
print(num_flights_test2019)

7250472

In [0]:
selected_test2019_withID.where(selected_test2019_withID["_id"] == 42+1).select("features", "label").show()

+--------------------+-----+
 features|label|
+--------------------+-----+
(85,[0,1,2,8,9,10...| 0.0|
+--------------------+-----+

In [0]:
from pyspark.sql.types import DoubleType
from pyspark.sql.functions import lit, udf

def ith_(v, i):
    try:
        return float(v[i])
    except ValueError:
        return None

# ith = udf(ith_, DoubleType())

# output = predictions.withColumn("del_prob",ith("probability", lit(1)))
# display(output)

In [0]:
val_predictions_3 = best_model_fold_3.transform(test_record)
val_predictions_3.columns

Out[31]: ['features', 'label', 'rawPrediction', 'prediction', 'probability']

In [0]:
## One example and it produces our 

test_record = selected_test2019_withID.where(selected_test2019_withID["_id"] == 0+1).select("features", "label")

id=0

# model 1-5
val_predictions_1 = best_model_fold_1.transform(test_record)
val_predictions_1 = val_predictions_1.select(["label","prediction","probability"]).withColumnRenamed("label","label_1")\
                                   .withColumnRenamed("prediction","prediction_1").withColumnRenamed("probability","probability_1").withColumn("_id",lit(id+1))

val_predictions_2 = best_model_fold_2.transform(test_record)
val_predictions_2 = val_predictions_2.select(["label","prediction","probability"]).withColumnRenamed("label","label_2")\
                                   .withColumnRenamed("prediction","prediction_2").withColumnRenamed("probability","probability_2").withColumn("_id",lit(id+1))

val_predictions_3 = best_model_fold_3.transform(test_record)
val_predictions_3 = val_predictions_3.select(["label","prediction","probability"]).withColumnRenamed("label","label_3")\
                                   .withColumnRenamed("prediction","prediction_3").withColumnRenamed("probability","probability_3").withColumn("_id",lit(id+1))

val_predictions_4 = best_model_fold_4.transform(test_record)
val_predictions_4 = val_predictions_4.select(["label","prediction","probability"]).withColumnRenamed("label","label_4")\
                                   .withColumnRenamed("prediction","prediction_4").withColumnRenamed("probability","probability_4").withColumn("_id",lit(id+1))

val_predictions_5 = best_model_fold_5.transform(test_record)
val_predictions_5 = val_predictions_5.select(["label","prediction","probability"]).withColumnRenamed("label","label_5")\
                                   .withColumnRenamed("prediction","prediction_5").withColumnRenamed("probability","probability_5").withColumn("_id",lit(id+1))

# chain horizontal joins
val_preds_all = val_predictions_1.join(val_predictions_2, ['_id']).join(val_predictions_3, ['_id']).join(val_predictions_4, ['_id']).join(val_predictions_5, ['_id'])

val_preds_all_big = val_preds_all

display(val_preds_all_big)

_id,label_1,prediction_1,probability_1,label_2,prediction_2,probability_2,label_3,prediction_3,probability_3,label_4,prediction_4,probability_4,label_5,prediction_5,probability_5
1,0.0,0.0,"Map(vectorType -> dense, length -> 2, values -> List(0.7685880064964294, 0.23141199350357056))",0.0,0.0,"Map(vectorType -> dense, length -> 2, values -> List(0.7439590692520142, 0.25604090094566345))",0.0,0.0,"Map(vectorType -> dense, length -> 2, values -> List(0.7285400629043579, 0.2714599370956421))",0.0,0.0,"Map(vectorType -> dense, length -> 2, values -> List(0.6941097974777222, 0.30589020252227783))",0.0,0.0,"Map(vectorType -> dense, length -> 2, values -> List(0.7341816425323486, 0.26581838726997375))"


In [0]:
# skip the first, already added
for id in range(1, num_flights_test2019):
    # get the record
    test_record = selected_test2019_withID.where(selected_test2019_withID["_id"] == id+1).select("features", "label").cache()
    
    if id%100000 == 0:
        print(f"On id: {id}")
    # get 5 predictions from each model
    
    # model 1-5
    val_predictions_1 = best_model_fold_1.transform(test_record)
    val_predictions_1 = val_predictions_1.select(["label","prediction","probability"]).withColumnRenamed("label","label_1")\
                                       .withColumnRenamed("prediction","prediction_1").withColumnRenamed("probability","probability_1").withColumn("_id",lit(id+1))
    
    val_predictions_2 = best_model_fold_2.transform(test_record)
    val_predictions_2 = val_predictions_2.select(["label","prediction","probability"]).withColumnRenamed("label","label_2")\
                                       .withColumnRenamed("prediction","prediction_2").withColumnRenamed("probability","probability_2").withColumn("_id",lit(id+1))
    
    val_predictions_3 = best_model_fold_3.transform(test_record)
    val_predictions_3 = val_predictions_3.select(["label","prediction","probability"]).withColumnRenamed("label","label_3")\
                                       .withColumnRenamed("prediction","prediction_3").withColumnRenamed("probability","probability_3").withColumn("_id",lit(id+1))
    
    val_predictions_4 = best_model_fold_4.transform(test_record)
    val_predictions_4 = val_predictions_4.select(["label","prediction","probability"]).withColumnRenamed("label","label_4")\
                                       .withColumnRenamed("prediction","prediction_4").withColumnRenamed("probability","probability_4").withColumn("_id",lit(id+1))
    
    val_predictions_5 = best_model_fold_5.transform(test_record)
    val_predictions_5 = val_predictions_5.select(["label","prediction","probability"]).withColumnRenamed("label","label_5")\
                                       .withColumnRenamed("prediction","prediction_5").withColumnRenamed("probability","probability_5").withColumn("_id",lit(id+1))
    
    # chain horizontal joins
    val_preds_all = val_predictions_1.join(val_predictions_2, ['_id']).join(val_predictions_3, ['_id']).join(val_predictions_4, ['_id']).join(val_predictions_5, ['_id'])
    
    # append or add row to a big dataframe
    val_preds_all_big = val_preds_all_big.union(val_preds_all)
        
    test_record.unpersist()
    

In [0]:
val_preds_all.write.parquet(f"{blob_url}/xgboost_predictions_preThresholding_4_9")

In [0]:
display(val_preds_all)